## Data Mining Competition Final Code 

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_columns', 30, 'max_rows', 30)

In [2]:
train = pd.read_csv('X_train.csv',encoding='cp949')
test = pd.read_csv('X_test.csv',encoding='cp949')
y_train = pd.read_csv('y_train.csv',encoding='cp949')

#####################################################
# feature 만든것 집어넣기 위해 빈 리스트 만듦.
feature = []

In [3]:
data = pd.concat([train,test])
data['sales_date'] = pd.to_datetime(data['sales_date'],format='%Y-%m-%d')
data['month'] = data['sales_date'].dt.month
data['weekday'] = data['sales_date'].dt.dayofweek
data['구매시간대'] = data['sales_time']//100 
data['할인율'] = round(data['dis_amt']/data['tot_amt'],2)
def fx(x):
    if 8<=x<=13:
        return('아침구매')
    elif 14<=x<=18:
        return('오후구매')
    else:
        return('밤구매')
data['구매시간'] = data['구매시간대'].map(fx)
data['pc_nm'] = data['pc_nm'].str.replace('영트렌디','영트랜디')
data['pc_nm'] = data['pc_nm'].str.replace('캐쥬얼','캐주얼')
data['pc_nm'] = data['pc_nm'].str.replace('트레디셔널','트래디셔널')
data['pc_nm'] = data['pc_nm'].str.replace('침구/수예','침구수예')
data['pc_nm'] = data['pc_nm'].str.replace('침구,수예','침구수예')
data['pc_nm'] = data['pc_nm'].str.replace('트랜디 케쥬얼','트래디셔널캐쥬얼')
data['buyer_nm'] = data['buyer_nm'].str.replace('행사장.*','행사장')
data['buyer_nm'] = data['buyer_nm'].str.replace('피혁A','피혁')
data['buyer_nm'] = data['buyer_nm'].str.replace('피혁B','피혁')
data.head()

,custid,sales_date,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon,inst_fee,month,weekday,구매시간대,할인율,구매시간
0,0,2000-06-25,1212,무역점,2116050008000,에스티로더,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,90000,9000,81000,3,0,6,6,12,0.1,아침구매
1,0,2000-06-25,1242,무역점,4125440008000,시슬리,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,39000,3900,35100,1,0,6,6,12,0.1,아침구매
2,0,2000-08-26,1810,본점,2116052008000,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,1,175000,17500,157500,3,0,8,5,18,0.1,오후구매
3,0,2000-08-26,1830,본점,4106430119900,듀퐁,수입의류,명품토탈,잡화파트,잡화가용팀,수입명품,1,455000,45500,409500,3,0,8,5,18,0.1,오후구매
4,0,2000-09-03,1802,무역점,2139141008000,랑콤,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,0,100000,10000,90000,3,0,9,6,18,0.1,오후구매


In [4]:
data.describe()

,custid,sales_time,goodcd,import_flg,tot_amt,dis_amt,net_amt,inst_mon,inst_fee,month,weekday,구매시간대,할인율
count,1.726430e+06,1.726430e+06,1.726430e+06,1.726430e+06,1.726430e+06,1.726430e+06,1.726430e+06,1.726430e+06,1.726430e+06,1.726430e+06,1.726430e+06,1.726430e+06,1.726430e+06
mean,2.490465e+04,1.544808e+03,3.916395e+12,1.346015e-01,9.866935e+04,3.353621e+03,9.531573e+04,1.870374e+00,2.688380e-02,6.533464e+00,3.482283e+00,1.518959e+01,2.702980e-02
std,1.446082e+04,2.461768e+02,9.259838e+11,3.412975e-01,3.152561e+05,1.563967e+04,3.048729e+05,1.282922e+00,1.617439e-01,3.379942e+00,1.935305e+00,2.477512e+00,3.140976e-02
min,0.000000e+00,0.000000e+00,2.101013e+12,0.000000e+00,-7.200000e+07,-1.479000e+06,-7.200000e+07,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.243700e+04,1.340000e+03,2.700000e+12,0.000000e+00,2.243200e+04,0.000000e+00,2.200000e+04,1.000000e+00,0.000000e+00,4.000000e+00,2.000000e+00,1.300000e+01,0.000000e+00
50%,2.489000e+04,1.550000e+03,4.202163e+12,0.000000e+00,5.200000e+04,0.000000e+00,5.044000e+04,1.000000e+00,0.000000e+00,6.000000e+00,4.000000e+00,1.500000e+01,0.000000e+00
75%,3.742700e+04,1.740000e+03,4.404830e+12,0.000000e+00,1.192000e+05,3.650000e+03,1.155000e+05,3.000000e+00,0.000000e+00,1.000000e+01,5.000000e+00,1.700000e+01,5.000000e-02
max,4.999400e+04,2.347000e+03,8.801192e+12,1.000000e+00,7.200000e+07,2.301440e+06,7.200000e+07,1.200000e+01,1.000000e+00,1.200000e+01,6.000000e+00,2.300000e+01,6.500000e-01


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1726430 entries, 0 to 689776
Data columns (total 22 columns):
custid        int64
sales_date    datetime64[ns]
sales_time    int64
str_nm        object
goodcd        int64
brd_nm        object
corner_nm     object
pc_nm         object
part_nm       object
team_nm       object
buyer_nm      object
import_flg    int64
tot_amt       int64
dis_amt       int64
net_amt       int64
inst_mon      int64
inst_fee      int64
month         int64
weekday       int64
구매시간대         int64
할인율           float64
구매시간          object
dtypes: datetime64[ns](1), float64(1), int64(12), object(8)
memory usage: 302.9+ MB


# Feautre 만들기 

## 총 구매금액

In [6]:
a = data.groupby('custid')['net_amt'].agg([('총구매금액',sum)]).reset_index()
feature.append(a)
a.head()

,custid,총구매금액
0,0,1567800
1,1,2715470
2,2,3495760
3,3,2208840
4,4,1023200


## 총 할인 금액

In [7]:
b = data.query("net_amt>0").groupby('custid')['dis_amt'].agg([('총할인금액',sum)]).reset_index()
b['총할인비율'] = b['총할인금액']/a['총구매금액']
feature.append(b)
b.head()

,custid,총할인금액,총할인비율
0,0,174200,0.111111
1,1,60280,0.022199
2,2,298540,0.085401
3,3,126440,0.057243
4,4,21800,0.021306


## 구매횟수

In [8]:
a = data.groupby('custid')['str_nm'].agg([('구매횟수','size')]).reset_index()
feature.append(a)
a.head()

,custid,구매횟수
0,0,11
1,1,26
2,2,11
3,3,30
4,4,4


## 내점일수, 내점당 평균구매액

In [9]:
a = data.query("tot_amt>0").groupby('custid')['sales_date'].nunique().reset_index()
a['내점당구매액']=round(data.query('tot_amt>0').groupby('custid')['net_amt'].agg([('내점당구매액',np.mean)]).reset_index()['내점당구매액']/a['sales_date'],2)
feature.append(a)
a.head()

,custid,sales_date,내점당구매액
0,0,7,20361.04
1,1,16,7343.28
2,2,7,84171.59
3,3,12,8540.65
4,4,2,127900.00


## 평균 할부개월수

In [10]:
a =data.query("net_amt>0").groupby('custid')['inst_mon'].agg([('평균할부개월수','mean')]).reset_index()
feature.append(a)
a.head()

,custid,평균할부개월수
0,0,2.818182
1,1,2.500000
2,2,3.444444
3,3,2.571429
4,4,4.500000


## 수입상품 구매비율

In [11]:
a = data.query('net_amt>0').groupby('custid')['import_flg'].agg([('수입상품구매',sum)]).reset_index()
a['수입상품구매비율'] = a['수입상품구매']/data.query('net_amt>0').groupby('custid')['import_flg'].agg([('a',np.size)]).reset_index()['a']
feature.append(a)
a.head()

,custid,수입상품구매,수입상품구매비율
0,0,7,0.636364
1,1,10,0.416667
2,2,1,0.111111
3,3,0,0.000000
4,4,1,0.250000


## 소분류 

In [12]:
a = pd.pivot_table(index='custid',columns='buyer_nm',values='tot_amt',aggfunc=np.size,fill_value=0,data=data.query("tot_amt>0")[['custid','buyer_nm','tot_amt']]).reset_index()
feature.append(a)
a.head()

buyer_nm,custid,가구,가전,기타바이어,니트단품,도자기크리스탈,디자이너부띠끄,문화완구,생활용품,섬유,수입명품,스포츠,엘레강스캐주얼,영캐주얼,유니캐주얼,유아동복,일반식품,장신구,점외,정장셔츠,조리식품,조리욕실,청과곡물,침구수예,캐릭터캐주얼,타운모피,트래디셔널캐주얼,피혁,행사장,화장품
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,8
1,1,0,1,0,2,0,0,0,0,1,6,1,1,3,0,2,0,1,0,0,0,1,0,0,2,0,0,0,0,3
2,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,2,1,0,1
3,3,0,1,0,1,0,0,0,0,1,0,8,0,2,4,1,3,0,0,2,0,1,0,0,1,0,1,1,0,1
4,4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1


In [13]:
a = pd.pivot_table(index='custid',columns='pc_nm',values='tot_amt',aggfunc=np.size,fill_value=0,data=data[['custid','pc_nm','tot_amt']]).reset_index()
feature.append(a)
a.head()

pc_nm,custid,(주)현스포츠아쌤,가구,가전,가전/문화,골프,남성잡화,남여구두,내의란제리,농산물,"니트,단품,모피",니트/단품,디자이너부띠끄,디자이너부띠크,디자이너부틱,...,축산물,침구수예,캐릭터정장,캐릭터캐주얼,캐주얼,타운란제리,트래디셔널,트래디셔널캐주얼,트래디셔널캐쥬얼,패션슈즈,패션시즌,피혁A,피혁B,핸드백,화장품
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,8
1,1,0,0,1,0,1,0,0,3,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,1,0,0,0,0,0,3
2,2,0,0,3,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,0,0,1,0,0,0,0,1
3,3,0,0,0,1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1


## 할인율별 구매횟수

In [14]:
a = pd.pivot_table(index='custid',columns='할인율',values='tot_amt',aggfunc=np.size,fill_value=0,data=data.query("tot_amt>0")[['custid','할인율','tot_amt']]).reset_index()
feature.append(a)
a.head()

할인율,custid,0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.2,0.4,0.65
0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0
1,1,15,0,0,0,0,9,0,0,0,0,0,0,0,0
2,2,2,0,0,0,0,0,0,0,0,0,7,0,0,0
3,3,10,0,0,1,1,15,0,0,0,0,1,0,0,0
4,4,2,0,0,0,0,2,0,0,0,0,0,0,0,0


In [15]:
a = a.drop('custid',axis=1).apply(lambda x: x/data.query("tot_amt>0").groupby('custid')['tot_amt'].size()).rename_axis('custid').reset_index()
feature.append(a)
a.head()

할인율,custid,0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.2,0.4,0.65
0,0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0
1,1,0.625000,0.0,0.0,0.000000,0.000000,0.375000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,2,0.222222,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.777778,0.0,0.0,0.0
3,3,0.357143,0.0,0.0,0.035714,0.035714,0.535714,0.0,0.0,0.0,0.0,0.035714,0.0,0.0,0.0
4,4,0.500000,0.0,0.0,0.000000,0.000000,0.500000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


## 구매시간별 횟수

In [16]:
a = pd.pivot_table(index='custid',columns='구매시간대',values='tot_amt',aggfunc=np.size,fill_value=0,data=data.query("tot_amt>0")[['custid','구매시간대','tot_amt']]).reset_index()
feature.append(a)
a.head()

구매시간대,custid,0,1,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0,0,0,0,0,0,0,2,0,0,0,0,0,4,5,0,0,0,0
1,1,0,0,0,0,1,2,2,0,1,4,6,3,4,0,1,0,0,0
2,2,0,0,0,0,1,1,0,1,1,0,1,1,1,2,0,0,0,0
3,3,0,0,0,0,0,2,3,2,3,0,0,5,11,2,0,0,0,0
4,4,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0


In [17]:
def fx3(x):
    if 9<=x<=17:
        return('퇴근전')
    else:
        return('퇴근후')
data['구매시간유형'] = data['구매시간대'].map(fx3)
a = pd.pivot_table(index='custid',columns='구매시간유형',values='tot_amt',aggfunc=np.size,
                   fill_value=0,data = data.query('tot_amt>0')[['custid','구매시간유형','tot_amt']]).reset_index()
feature.append(a)
a.head()

구매시간유형,custid,퇴근전,퇴근후
0,0,2,9
1,1,19,5
2,2,6,3
3,3,15,13
4,4,4,0


In [18]:
a = a.drop('custid',axis=1).apply(lambda x: x/data.query("tot_amt>0").groupby('custid')['tot_amt'].size()).rename_axis('custid').reset_index()
feature.append(a)
a.head()

구매시간유형,custid,퇴근전,퇴근후
0,0,0.181818,0.818182
1,1,0.791667,0.208333
2,2,0.666667,0.333333
3,3,0.535714,0.464286
4,4,1.000000,0.000000


In [19]:
a = pd.pivot_table(index='custid',columns='구매시간',values='tot_amt',aggfunc=np.size,fill_value=0,data=data.query("tot_amt>0")[['custid','tot_amt','구매시간']]).reset_index()
feature.append(a)
a.head()

구매시간,custid,밤구매,아침구매,오후구매
0,0,5,2,4
1,1,1,5,18
2,2,2,3,4
3,3,2,7,19
4,4,0,0,4


## 평균 환불금액

In [20]:
a = data.query("net_amt<0").groupby('custid')['net_amt'].agg([('환불횟수',np.size),('평균환불금액',np.mean)]).reset_index()
a['평균환불금액'] = abs(a['평균환불금액'])
a['반품율'] = round(a['환불횟수']/data.query("net_amt>0").groupby('custid')['net_amt'].size(),2)
feature.append(a)
a.head()

,custid,환불횟수,평균환불금액,반품율
0,1,2,52175.0,0.18
1,2,2,903525.0,0.08
2,3,2,330410.0,0.22
3,6,2,31250.0,0.07
4,8,2,84000.0,0.50


## 계절별 구매 비율

In [21]:
def f1(x):
    if 3 <= x <= 5 :
        return('봄_구매건수')
    elif 6 <= x <= 8 :
        return('여름_구매건수')
    elif 9 <= x <= 11 :    
        return('가을_구매건수')
    else :
        return('겨울_구매건수')    
    
data['season'] = data['month'].apply(f1)
f = pd.pivot_table(data.query("tot_amt>0")[['custid','season','tot_amt']], index='custid', columns='season',values='tot_amt',
                   aggfunc=np.size, fill_value=0).reset_index()
feature.append(f); f

season,custid,가을_구매건수,겨울_구매건수,봄_구매건수,여름_구매건수
0,0,3,3,1,4
1,1,6,1,10,7
2,2,5,2,0,2
3,3,3,6,8,11
4,4,0,0,0,4
5,5,12,1,6,13
6,6,13,2,8,6
7,7,7,12,1,15
8,8,4,2,1,9
9,9,18,11,11,16


In [22]:
def f2(x):
    if x <= 3 :
        return('월화수목_구매건수')
    else :
        return('금토일_구매건수')    
    
data['요일2'] = data['weekday'].map(f2)
f = pd.pivot_table(data.query("tot_amt>0")[['custid','요일2','tot_amt']], index='custid', columns='요일2', values='tot_amt',
                   aggfunc=np.size, fill_value=0).reset_index()
f['금토일_구매비율'] =round(f['금토일_구매건수']/data.query("tot_amt>0").groupby('custid')['tot_amt'].size(),2)
f['월화수목_구매비율'] =round(f['월화수목_구매건수']/data.query("tot_amt>0").groupby('custid')['tot_amt'].size(),2)

feature.append(f); f

요일2,custid,금토일_구매건수,월화수목_구매건수,금토일_구매비율,월화수목_구매비율
0,0,9,2,0.82,0.18
1,1,16,8,0.67,0.33
2,2,5,4,0.56,0.44
3,3,16,12,0.57,0.43
4,4,1,3,0.25,0.75
5,5,14,18,0.44,0.56
6,6,18,11,0.62,0.38
7,7,17,18,0.49,0.51
8,8,13,3,0.81,0.19
9,9,30,26,0.54,0.46


## 화장품 구매비율

In [23]:
a = data.query("pc_nm == '화장품' and net_amt>0").groupby('custid')['net_amt'].agg([('화장품구매','size'),('화장품구매금액','sum')]).reset_index()
b = data.query("net_amt>0").groupby('custid')['net_amt'].agg([('구매금액','sum'),('구매횟수','size')]).reset_index()
c = pd.merge(b,a,how='left',on='custid').fillna(0)
c['화장품구매비율'] = c['화장품구매']/c['구매횟수']
c['화장품금액비율'] = c['화장품구매금액']/c['구매금액']
a = c[['custid','화장품구매비율','화장품금액비율']]
feature.append(a)
a.head()

,custid,화장품구매비율,화장품금액비율
0,0,0.727273,0.606200
1,1,0.125000,0.056262
2,2,0.111111,0.024779
3,3,0.035714,0.061906
4,4,0.250000,0.055708


In [24]:
X_train = DataFrame({'custid': train.custid.unique()})
for f in feature :
    X_train = pd.merge(X_train, f, how='left',on='custid').fillna(0)
print(X_train)

X_test = DataFrame({'custid': test.custid.unique()})
for f in feature :
    X_test = pd.merge(X_test, f, how='left',on='custid').fillna(0)
IDtest = X_test.custid
print(X_test)

       custid     총구매금액   총할인금액     총할인비율  구매횟수  sales_date     내점당구매액  \
0           0   1567800  174200  0.111111    11           7   20361.04   
1           1   2715470   60280  0.022199    26          16    7343.28   
2           2   3495760  298540  0.085401    11           7   84171.59   
3           3   2208840  126440  0.057243    30          12    8540.65   
4           4   1023200   21800  0.021306     4           2  127900.00   
5           5   4692499  361260  0.076987    32          21    6982.89   
6           6   3469929  315100  0.090809    31          10   12180.79   
7           7   1201252   21930  0.018256    35          23    1492.24   
8           8   1242480   25020  0.020137    18           9    9795.00   
9           9   4742770  235270  0.049606    59          33    2786.66   
10         10   1298340   56830  0.043771    24          15    4159.83   
11         11   7049869  147080  0.020863    66          42    2742.09   
12         12   2501677   97750  0.039

       custid     총구매금액   총할인금액     총할인비율  구매횟수  sales_date     내점당구매액  \
0       30000   2000460   85680  0.042830    27          15    5514.26   
1       30001   4040250  121310  0.030025    27          10   15775.81   
2       30002   7692206  359700  0.046762   100          28    3364.19   
3       30003   1348730   22190  0.016453    43          25    1585.16   
4       30004   2883691    8280  0.002871    55          32    1833.39   
5       30005     54150    2850  0.052632     1           1   54150.00   
6       30006    572100   17650  0.030851     7           5   16345.71   
7       30007    285400   10100  0.035389     6           4   11891.67   
8       30008    393270    6950  0.017672     9           7    9865.54   
9       30009    234900    7100  0.030226     4           2   48650.00   
10      30010   1518500   43100  0.028383    11           4   34511.36   
11      30011   1660257   56620  0.034103    31          16    3503.66   
12      30012   2646500  199300  0.075

## GradientBoosting Classifier

In [25]:
import scipy as sp
from scipy.stats import randint as sp_randint
import scipy.stats
from sklearn.model_selection import RandomizedSearchCV

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

parameters = {
    'loss':['deviance','exponential'],
    "learning_rate": sp.stats.uniform(),
    "min_samples_split": sp.stats.uniform(),
    "max_depth":sp_randint(5,30),
    "subsample":sp.stats.uniform(),
    "n_estimators":sp_randint(5,40)
    }
gbc = GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.5671441276277093, loss='deviance',
              max_depth=15, max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=0.436747263026799,
              min_weight_fraction_leaf=0.0, n_estimators=14,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=0.5356041734976563, tol=0.0001,
              validation_fraction=0.1, verbose=0, warm_start=False)
model = GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1289262976548533, loss='deviance',
              max_depth=8, max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=0.3637107709426226,
              min_weight_fraction_leaf=0.0, n_estimators=25,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=0.634274057957335, tol=0.0001,
              validation_fraction=0.1, verbose=0, warm_start=False)
model.fit(X_train.drop('custid',axis=1),y_train['gender'])
pred0 = model.predict_proba(X_test.drop('custid',axis=1))[:,1]

## XGB Classifier

In [27]:
from xgboost import XGBClassifier

parameters = {
    "learning_rate": sp.stats.uniform(),
    "max_depth":sp_randint(5,30),
    "subsample":sp.stats.uniform(),
    "n_estimators":sp_randint(5,40)
    }
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.06528650438687811,
       max_delta_step=0, max_depth=24, min_child_weight=1, missing=None,
       n_estimators=36, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.1817270377811787)
model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.009356704856532616,
       max_delta_step=0, max_depth=7, min_child_weight=1, missing=None,
       n_estimators=21, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.7163272041185655)
model.fit(X_train.drop('custid',axis=1),y_train['gender'])
pred1 = model.predict_proba(X_test.drop('custid',axis=1))[:,1]

## LGBM Classifier

In [28]:
from lightgbm import LGBMClassifier

parameters = {
    "learning_rate": sp.stats.uniform(),
    "max_depth":sp_randint(5,30),
    "subsample":sp.stats.uniform(),
    "n_estimators":sp_randint(5,40)
    }
lgbm = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.06528650438687811,
        max_depth=24, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=36, n_jobs=-1, num_leaves=31,
        objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=0.1817270377811787,
        subsample_for_bin=200000, subsample_freq=0)
model = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.15896958364551972,
        max_depth=18, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=26, n_jobs=-1, num_leaves=31,
        objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=0.3172017420692961,
        subsample_for_bin=200000, subsample_freq=0)
model.fit(X_train.drop('custid',axis=1),y_train['gender'])
pred2 = model.predict_proba(X_test.drop('custid',axis=1).values)[:,1]

In [29]:
pred_a = (pred1*pred2*pred0)**(1/3)

In [30]:
from sklearn.ensemble import VotingClassifier
votingC = VotingClassifier(estimators=[('gbc', gbc),('xgb', xgb),('lgbm',lgbm)], voting='soft')
pred_b = votingC.fit(X_train, y_train['gender']).predict_proba(X_test)[:,1]

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_columns', 30, 'max_rows', 10)

In [32]:
tr_train = pd.read_csv('X_train.csv', encoding='cp949')
tr_test = pd.read_csv('X_test.csv', encoding='cp949')
tr = pd.concat([tr_train, tr_test])
tr['real_amt']= tr.tot_amt / tr.inst_mon
tr['sdate'] = tr.sales_date.str[:10]
tr['dist_rate'] = (tr['dis_amt']/tr['tot_amt'])*100
tr.loc[456697,"sales_time"] = 1530
tr["time"] = tr['sdate'].astype(str).str.cat(tr["sales_time"].astype(str), sep =" ")
tr["time"] = pd.to_datetime(tr.time, format='%Y-%m-%d %H%M')
features = []

In [33]:
#평균할인율
f = tr.groupby('custid')['dist_rate'].agg([('dis_rate', 'mean')]).reset_index()
features.append(f)
f.head(1)

,custid,dis_rate
0,0,10.0


In [34]:
#평균시간
f = tr.groupby(['custid'])['sales_time'].agg([('sales_time', 'mean')]).reset_index()
features.append(f)
f.head(1)

,custid,sales_time
0,0,1757.090909


In [35]:
#남성파트
df = tr.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df['part_nm'] = np.where(df.part_nm.str.contains('남성'), '남성', '비남성')
df = df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
df['남성part'] = (df['남성'] / (df['남성'] + df['비남성'])) * 100
df = df.fillna(0)
features.append(df)
f.head(1)

,custid,sales_time
0,0,1757.090909


In [36]:
#화장품구매비율
df = tr.groupby(['custid','corner_nm'])['tot_amt'].agg([('tot_amt_corner', 'sum')]).reset_index()
df['corner_nm'] = np.where(df.corner_nm.str.contains('화장품'), '화장품', '비화장품')
df = df.pivot_table(values='tot_amt_corner', index=df.custid, columns='corner_nm', aggfunc='first',fill_value=0).reset_index()
df['화장품비율'] = (df['화장품'] / (df['화장품'] + df['비화장품'])) * 100
df = df.fillna(0)
features.append(df)
f.head(1)

,custid,sales_time
0,0,1757.090909


In [37]:
#쇼핑시간
df = tr.groupby(['sdate','custid'])['time'].agg([('time', ['min','max'])]).reset_index()
df['shopping_time'] = (df['time']['max'] - df['time']['min']).dt.total_seconds()
df.drop(['sdate','time'], axis=1, inplace=True,level=0)
df = df.groupby(['custid'])['shopping_time'].agg([('shopping_time_mean','mean')]).reset_index()
features.append(df)
f.head(1)

,custid,sales_time
0,0,1757.090909


In [38]:
#할부대비평균실구매
f = tr.groupby('custid')['real_amt'].agg([('real_amt', 'mean')]).reset_index()
features.append(f)
f.head(1)

,custid,real_amt
0,0,55151.515152


In [39]:
#평균구매상품종류
df =tr.groupby(['custid','goodcd'])['tot_amt'].agg([('good_count', 'count')]).reset_index()
f = df.groupby(['custid'])['good_count'].agg([('good_count_mean', 'mean')]).reset_index()
features.append(f)
f.head(1)

,custid,good_count_mean
0,0,1.571429


In [40]:
#지역
df = tr.groupby(['custid','str_nm'])['tot_amt'].agg([('tot_amt_str', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_str', index=df.custid, columns='str_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)
f.head(1)

,custid,good_count_mean
0,0,1.571429


In [41]:
#팀별
df = tr.groupby(['custid','team_nm'])['tot_amt'].agg([('tot_amt_team', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_team', index=df.custid, columns='team_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)
f.head(1)

,custid,good_count_mean
0,0,1.571429


In [42]:
#총구매수입상품
df = tr.groupby(['custid'])['import_flg'].agg([('import_flg_sum', 'sum')]).reset_index()
features.append(df)
f.head(1)

,custid,good_count_mean
0,0,1.571429


In [43]:
#월별총구매수입상품
df = tr.groupby(['custid'])['import_flg'].agg([('inst_mon_sum', 'sum')]).reset_index()
features.append(df)
f.head(1)

,custid,good_count_mean
0,0,1.571429


In [44]:
#파트
df = tr.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)
f.head(1)

,custid,good_count_mean
0,0,1.571429


In [45]:
#코너
df = tr.groupby(['custid','corner_nm'])['tot_amt'].agg([('tot_amt_corner', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_corner', index=df.custid, columns='corner_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)
f.head(1)

,custid,good_count_mean
0,0,1.571429


In [46]:
#pc
df = tr.groupby(['custid','pc_nm'])['tot_amt'].agg([('tot_amt_pc', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_pc', index=df.custid, columns='pc_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)
f.head(1)

,custid,good_count_mean
0,0,1.571429


In [47]:
#브랜드
df = tr.groupby(['custid','brd_nm'])['tot_amt'].agg([('tot_amt_brd', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_brd', index=df.custid, columns='brd_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)
f.head(1)

,custid,good_count_mean
0,0,1.571429


In [48]:
#구매자
df = tr.groupby(['custid','buyer_nm'])['tot_amt'].agg([('tot_amt_buyer', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_buyer', index=df.custid, columns='buyer_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)
f.head(1)

,custid,good_count_mean
0,0,1.571429


In [49]:
#구매시간
from datetime import timedelta as dt
test = tr.groupby(['custid'])['sales_date'].agg([('sales_date', 'max')]).reset_index()
test['days'] = (pd.to_datetime('2002-01-01') - pd.to_datetime(test.sales_date)).dt.days
test.drop(['sales_date'], axis=1, inplace=True)
features.append(test)
test.head(1)

,custid,days
0,0,267


In [50]:
#일평균구매액
test2 = tr.groupby(['sales_date','custid'])['tot_amt'].agg([('day_amt', 'sum')]).reset_index()
test2 = test2.groupby(['custid'])['day_amt'].agg([('일평균구매액', 'mean')]).reset_index()
features.append(test2)
test2.head(1)

,custid,일평균구매액
0,0,248857.142857


In [51]:
#일평균구매건
df = tr.groupby(['sales_date','custid'])['custid'].agg([('day_visit', 'count')]).reset_index()
f = df.groupby(['custid'])['day_visit'].agg([('일평균구매건', 'mean')]).reset_index()
features.append(f)
f.head(1)

,custid,일평균구매건
0,0,1.571429


In [52]:
#총구매액
f = tr.groupby('custid')['tot_amt'].agg([('총구매액', 'sum')]).reset_index()
features.append(f)
f.head(1)

,custid,총구매액
0,0,1742000


In [53]:
#구매건수
f = tr.groupby('custid')['tot_amt'].agg([('구매건수', 'size')]).reset_index()
features.append(f)
f.head(1)

,custid,구매건수
0,0,11


In [54]:
#평균 구매가격
f = tr.groupby('custid')['tot_amt'].agg([('평균구매가격', 'mean')]).reset_index()
features.append(f)
f.head(1)

,custid,평균구매가격
0,0,158363.636364


In [55]:
#평균 할부개월수 
f = tr.groupby('custid')['inst_mon'].agg([('평균할부개월수', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f)
f.head(1)

,custid,평균할부개월수
0,0,2.8


In [56]:
# 구매상품다양성
n = tr.corner_nm.nunique()
f = tr.groupby('custid')['brd_nm'].agg([('구매상품다양성', lambda x: len(x.unique()) / n)]).reset_index()
features.append(f)
f.head(1)

,custid,구매상품다양성
0,0,0.022654


In [57]:
# 내점일수
tr['sdate'] = tr.sales_date.str[:10]
f = tr.groupby(by = 'custid')['sdate'].agg([('내점일수','nunique')]).reset_index()
features.append(f)
f.head(1)

,custid,내점일수
0,0,7


In [58]:
#수입상품 구매비율
x = tr[tr['import_flg'] == 1].groupby('custid').size() / tr.groupby('custid').size()
f = x.reset_index().rename(columns={0: '수입상품_구매비율'}).fillna(0)
f.iloc[:,1] = (f.iloc[:,1]*100).apply(round, args=(1,))
features.append(f)
f.head(1)

,custid,수입상품_구매비율
0,0,63.6


In [59]:
# 요일구매패턴
def fw(x):
    k = x.dayofweek
    if k <= 4 :
        return('주중_방문')
    else :
        return('주말_방문')    
    
df = tr.copy()
df = df.drop_duplicates(['custid','sales_date'])

df['week'] = pd.to_datetime(df.sales_date).apply(fw)
df = pd.pivot_table(df, index='custid', columns='week', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
df['주말방문비율'] = ((df.iloc[:,1] / (df.iloc[:,1]+df.iloc[:,2]))*100).apply(round, args=(1,))
f = df.copy().iloc[:,[0,-1]]
features.append(f)
f.head(1)

week,custid,주말방문비율
0,0,42.9


In [60]:
# 계절별 구매건수
def f1(x):
    k = x.month
    if 3 <= k <= 5 :
        return('봄-구매건수')
    elif 6 <= k <= 8 :
        return('여름-구매건수')
    elif 9 <= k <= 11 :    
        return('가을-구매건수')
    else :
        return('겨울-구매건수')    
    
tr['season'] = pd.to_datetime(tr.sales_date).apply(f1)
f = pd.pivot_table(tr, index='custid', columns='season', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f)
f.head(1)

season,custid,가을-구매건수,겨울-구매건수,봄-구매건수,여름-구매건수
0,0,3,3,1,4


In [61]:
# 시간대별 구매건수
def f2(x):
    if 9 <= x <= 12 :
        return('아침_구매건수')
    elif 13 <= x <= 17 :
        return('점심_구매건수')
    else :
        return('저녁_구매건수')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리

tr['timeslot'] = tr.sales_date.str.split(' |:', expand=True).iloc[:,1].astype(int).apply(f2)
f = pd.pivot_table(tr, index='custid', columns='timeslot', values='tot_amt', 
                   aggfunc=np.size).reset_index()
features.append(f)
f.head(1)

timeslot,custid,저녁_구매건수
0,0,11


In [62]:
#주구매코너
f = tr.groupby('custid')['corner_nm'].agg([('주구매코너', lambda x: x.value_counts().index[0])]).reset_index()
f = pd.get_dummies(f, columns=['주구매코너'])  # This method performs One-hot-encoding
features.append(f)
f.head(1)

,custid,주구매코너_DC캐주얼,주구매코너_GBR 지원,주구매코너_L/B침구,주구매코너_N/B침구,주구매코너_NB제화,주구매코너_NB핸드백,주구매코너_TOP디자이너,"주구매코너_TV,VTR",주구매코너_TV.VTR,주구매코너_TV/VCR,주구매코너_TV/VTR,주구매코너_가방,주구매코너_가스기기,주구매코너_가전특정,...,주구매코너_핸드백,주구매코너_핸드백임대,주구매코너_핸드백행사,주구매코너_행사,주구매코너_행사/단품,주구매코너_행사슈즈,주구매코너_행사핸드백,주구매코너_향수,주구매코너_헤어ACC,주구매코너_헤어악세사리,주구매코너_헤어액세사리,주구매코너_홈데코,주구매코너_화장잡화,주구매코너_화장품,주구매코너_훼미닌부틱
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
tr['구매시간대'] =tr['sales_time']//100 
tr['구매시간대_시'] = str(tr['구매시간대']) + '시'
a = pd.pivot_table(index='custid',columns='구매시간대',values='tot_amt',aggfunc=np.size,fill_value=0,data=tr.query("tot_amt>0")[['custid','구매시간대','tot_amt']]).reset_index()
features.append(a)
a.head(1)

구매시간대,custid,1,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0,0,0,0,0,0,2,0,0,0,0,0,4,5,0,0,0,0


In [64]:
b = tr.query("net_amt>0").groupby('custid')['net_amt'].agg([('구매횟수','size')]).reset_index()
a = a.drop('custid',axis=1).apply(lambda x: x/b['구매횟수']).rename_axis('custid').reset_index()
features.append(a)
a.head(1)

구매시간대,custid,1,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0,0.0,0.0,0.0,0.0,0.0,0.181818,0.0,0.0,0.0,0.0,0.0,0.363636,0.454545,0.0,0.0,0.0,0.0


In [65]:
tr['pc_nm1'] = tr['pc_nm'] +'구분'
a = pd.pivot_table(index='custid',columns='pc_nm',values='tot_amt',aggfunc=np.size,fill_value=0,data=tr[['custid','pc_nm','tot_amt']]).reset_index()
features.append(a)
a.head(1)

pc_nm,custid,(주)현스포츠아쌤,가구,가전,가전/문화,골프,남성잡화,남여구두,내의란제리,농산물,"니트,단품,모피",니트/단품,디자이너부띠끄,디자이너부띠크,디자이너부틱,...,캐릭터캐주얼,캐릭터캐쥬얼,캐주얼,캐쥬얼,타운란제리,트래디셔널,트래디셔널캐쥬얼,트랜디 케쥬얼,트레디셔널,패션슈즈,패션시즌,피혁A,피혁B,핸드백,화장품
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,8


In [66]:
a = a.iloc[:,1:].apply(lambda x:x/b['구매횟수']).rename_axis('custid').reset_index()
features.append(a)
a.head(1)

pc_nm,custid,(주)현스포츠아쌤,가구,가전,가전/문화,골프,남성잡화,남여구두,내의란제리,농산물,"니트,단품,모피",니트/단품,디자이너부띠끄,디자이너부띠크,디자이너부틱,...,캐릭터캐주얼,캐릭터캐쥬얼,캐주얼,캐쥬얼,타운란제리,트래디셔널,트래디셔널캐쥬얼,트랜디 케쥬얼,트레디셔널,패션슈즈,패션시즌,피혁A,피혁B,핸드백,화장품
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.727273


In [67]:
def fx3(x):
    if 9<=x<=17:
        return('퇴근전')
    else:
        return('퇴근후')
tr['구매시간유형'] = tr['구매시간대'].map(fx3)
a = pd.pivot_table(index='custid',columns='구매시간유형',values='tot_amt',aggfunc=np.size,
                   fill_value=0,data = tr.query('tot_amt>0')[['custid','구매시간유형','tot_amt']]).reset_index()
features.append(a)
a.head(1)

구매시간유형,custid,퇴근전,퇴근후
0,0,2,9


In [68]:
a = a.drop('custid',axis=1).apply(lambda x: x/tr.query("tot_amt>0").groupby('custid')['tot_amt'].size()).rename_axis('custid').reset_index()
features.append(a)
a.head(1)

구매시간유형,custid,퇴근전,퇴근후
0,0,0.181818,0.818182


In [69]:
a = tr.query("net_amt<0").groupby('custid')['net_amt'].agg([('환불횟수',np.size),('평균환불금액',np.mean)]).reset_index()
a['평균환불금액'] = abs(a['평균환불금액'])
a['반품율'] = round(a['환불횟수']/tr.query("net_amt>0").groupby('custid')['net_amt'].size(),2)
features.append(a)
a.head()

,custid,환불횟수,평균환불금액,반품율
0,1,2,52175.0,0.18
1,2,2,903525.0,0.08
2,3,2,330410.0,0.22
3,6,2,31250.0,0.07
4,8,2,84000.0,0.50


In [70]:
# Train Test data 만들기 
X_train = pd.DataFrame({'custid': tr_train.custid.unique()})
for f in features :
    X_train = pd.merge(X_train, f, how='left',on='custid').fillna(0)
display(X_train.shape)

X_test = pd.DataFrame({'custid': tr_test.custid.unique()})
for f in features :
    X_test = pd.merge(X_test, f, how='left',on='custid').fillna(0)
display(X_test.shape)

(30000, 2874)

(19995, 2874)

In [71]:
X_train['평균내점구매액'] = X_train['총구매액']/X_train['내점일수']
X_train['주중방문비율'] = (100 - X_train['주말방문비율'])
X_train['국내상품_구매비율'] = (100 - X_train['수입상품_구매비율'])
X_train['할부구매가격'] = X_train['평균구매가격'] / X_train['평균할부개월수']
X_train['구매상품다양성'] = X_train['총구매액'] / X_train['구매상품다양성']
X_train['주말방문수'] = (X_train['주말방문비율'] * X_train['내점일수']) / 100
X_train['주말방문수'] = X_train['주말방문수'].astype('int64')
X_train['주중방문수'] = X_train['내점일수'] - X_train['주말방문수']
X_train['주중방문수'] = X_train['주중방문수'].astype('int64')
X_train['내점당편균구매건수'] = X_train['구매건수']/X_train['내점일수']
X_train['주중구매액'] = X_train['총구매액']*(X_train['주중방문비율']/100)
X_train['주말구매액'] = X_train['총구매액'] - X_train['주중구매액']

In [72]:
X_test['평균내점구매액'] = X_test['총구매액']/X_test['내점일수']
X_test['주중방문비율'] = (100 - X_test['주말방문비율'])
X_test['국내상품_구매비율'] = (100 - X_test['수입상품_구매비율'])
X_test['할부구매가격'] = X_test['평균구매가격'] / X_test['평균할부개월수']
X_test['구매상품다양성'] = X_test['총구매액'] / X_test['구매상품다양성']
X_test['주말방문수'] = (X_test['주말방문비율'] * X_test['내점일수']) / 100
X_test['주말방문수'] = X_test['주말방문수'].astype('int64')
X_test['주중방문수'] = X_test['내점일수'] - X_test['주말방문수']
X_test['주중방문수'] = X_test['주중방문수'].astype('int64')
X_test['내점당편균구매건수'] = X_test['구매건수']/X_test['내점일수']
X_test['주중구매액'] = X_test['총구매액']*(X_test['주중방문비율']/100)
X_test['주말구매액'] = X_test['총구매액'] - X_test['주중구매액']

In [73]:
IDtest = X_test.custid;
X_train.drop(['custid'], axis=1, inplace=True)
X_test.drop(['custid'], axis=1, inplace=True)
y_train = pd.read_csv('y_train.csv').gender

In [74]:
X_train.clip(lower=0,inplace=True)

In [75]:
X_train.columns = np.arange(0,len(X_train.columns))
X_test.columns = np.arange(0,len(X_train.columns))
max_features = X_train.shape[1]

In [76]:
GBC = GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1289262976548533, loss='deviance',
              max_depth=8, max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=0.3637107709426226,
              min_weight_fraction_leaf=0.0, n_estimators=25,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=0.634274057957335, tol=0.0001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [77]:
clf = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.009356704856532616,
       max_delta_step=0, max_depth=7, min_child_weight=1, missing=None,
       n_estimators=21, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.7163272041185655)

In [78]:
lgbm = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.06528650438687811,
        max_depth=24, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=36, n_jobs=-1, num_leaves=31,
        objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=0.1817270377811787,
        subsample_for_bin=200000, subsample_freq=0)

In [79]:
from sklearn.ensemble import VotingClassifier
votingC = VotingClassifier(estimators=[('clf', clf), ('gbc', GBC)], voting='soft', n_jobs=-1)

In [80]:
pred = votingC.fit(X_train, y_train).predict_proba(X_test)[:,1]

In [81]:
df = tr.groupby(['custid'])['sales_time'].agg([('sales_time', 'mean')]).reset_index()

In [82]:
def f4(x):
    if x <= 978 :
        return('st_cat01')
    elif 979 <= x <= 1464 :
        return('st_cat02')
    elif 1465 <= x <= 1564 :
        return('st_cat03')
    elif 1565 <= x <= 1658 :
        return('st_cat04')
    else :
        return('st_cat05')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리
df['sale_ct'] = df.sales_time.apply(f4)
df.drop(['sales_time'], axis=1, inplace=True)
df = pd.get_dummies(df, columns=['sale_ct'])

In [83]:
train_1 = df.query('custid not in @IDtest').drop(columns=['custid'])
test_1 = df.query('custid in @IDtest').drop(columns=['custid'])

In [84]:
df = tr.groupby(['custid'])['dist_rate'].agg([('dist_amt', 'mean')]).reset_index()

In [85]:
def f3(x):
    if x <= 0 :
        return('cat01')
    elif 1 <= x <= 1.83 :
        return('cat02')
    elif 1.84 <= x <= 2.83 :
        return('cat03')
    elif 2.84 <= x <= 3.92 :
        return('cat04')
    else :
        return('cat05')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리
df['dist'] = df.dist_amt.apply(f3)
df.drop(['dist_amt'], axis=1, inplace=True)
df = pd.get_dummies(df, columns=['dist'])

In [86]:
train_2 = df.query('custid not in @IDtest').drop(columns=['custid'])
test_2 = df.query('custid in @IDtest').drop(columns=['custid'])

In [87]:
def f2(x):
    k = x.hour
    if 9 <= k <= 12 :
        return('아침_구매건수')
    elif 13 <= k <= 17 :
        return('점심_구매건수')
    else :
        return('저녁_구매건수')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리

tr['timeslot'] = tr.time.apply(f2)

In [88]:
def f1(x):
    k = x.month
    if 3 <= k <= 5 :
        return('봄-구매건수')
    elif 6 <= k <= 8 :
        return('여름-구매건수')
    elif 9 <= k <= 11 :    
        return('가을-구매건수')
    else :
        return('겨울-구매건수')    
    
tr['season'] = tr.time.apply(f1)

In [89]:
def fw(x):
    k = x.dayofweek
    if k <= 4 :
        return('주중_방문')
    else :
        return('주말_방문')    
    
tr['week'] = tr.time.apply(fw)

In [ ]:
tr['sales_hour'] = tr['sales_time']//100;
tr['sales_wkday'] = pd.to_datetime(tr.sales_date).dt.weekday

def makeBOW(col):
    
    f = lambda x: np.where(len(x) >=1, 1, 0)

    train = pd.pivot_table(tr, index='custid', columns=col, values='tot_amt',
                             aggfunc=f, fill_value=0).reset_index(). \
                             query('custid not in @IDtest').drop(columns=['custid'])
    test = pd.pivot_table(tr, index='custid', columns=col, values='tot_amt',
                             aggfunc=f, fill_value=0).reset_index(). \
                             query('custid in @IDtest').drop(columns=['custid'])
    return train, test
train1, test1 = makeBOW('brd_nm')
train2, test2 = makeBOW('corner_nm')
train3, test3 = makeBOW('sales_hour')
train4, test4 = makeBOW('sales_wkday')
train5, test5 = makeBOW('timeslot')
train6, test6 = makeBOW('week')
train7, test7 = makeBOW('season')
#train6, test6 = makeBOW('dis_rate_ca')
X_train = pd.concat([train1, train2, train3,train4,train5,train6,train7,train_1,train_2], axis=1).values
X_test = pd.concat([test1, test2, test3,test4,test5,test6,test7,test_1,test_2], axis=1).values
max_features = X_train.shape[1]

In [ ]:
from keras import models
from keras import layers
from keras.optimizers import RMSprop
from keras import regularizers
from keras.callbacks import EarlyStopping

model3 = models.Sequential()
model3.add(layers.Dense(1, input_shape=(max_features,), kernel_regularizer=regularizers.l2(0.01)))
model3.add(layers.Activation('sigmoid'))

model3.summary()

model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model3.fit(X_train, y_train, epochs=100, batch_size=64, 
                    validation_split=0.2, callbacks=[EarlyStopping(patience=5)])

In [ ]:
# 확률로 나오니까 첫번째 columns의 값만 가져오기 위해 만듦. 
pred_nn = model3.predict(X_test)[:,0]

In [ ]:
pred_c = (1/2*pred)+(1/2*pred_nn)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
tr_train = pd.read_csv('X_train.csv', encoding='cp949')
tr_test = pd.read_csv('X_test.csv', encoding='cp949')
yy_train = pd.read_csv('y_train.csv', encoding='cp949')
tr = pd.concat([tr_train, tr_test])
features = []
tr.head(2)

In [ ]:
newfeat = tr_train.merge(yy_train, how='left')
newfeat.head(2)

In [ ]:
f = newfeat[['custid','str_nm','gender']].set_index('custid')
f= f.groupby('str_nm').agg('mean')
f = f.to_dict()['gender']
tr['str_nm_target'] = tr.str_nm.apply(lambda x: f.get(x,0))
f = tr.groupby('custid')['str_nm_target'].agg({'str_nm_tar_mean':'mean'}).reset_index()
features.append(f); f.head(2)

In [ ]:
f = newfeat[['custid','pc_nm','gender']].set_index('custid')
f= f.groupby('pc_nm').agg('mean')
f = f.to_dict()['gender']
tr['pc_nm_target'] = tr.pc_nm.apply(lambda x: f.get(x,0))
f = tr.groupby('custid')['pc_nm_target'].agg({'pc_nm_tar_mean':'mean'}).reset_index()
features.append(f); f.head(2)

In [ ]:
f = newfeat[['custid','part_nm','gender']].set_index('custid')
f= f.groupby('part_nm').agg('mean')
f = f.to_dict()['gender']
tr['part_nm_target'] = tr.part_nm.apply(lambda x: f.get(x,0))
f = tr.groupby('custid')['part_nm_target'].agg({'part_nm_tar_mean':'mean'}).reset_index()
features.append(f); f.head(2)

In [ ]:
f = newfeat[['custid','buyer_nm','gender']].set_index('custid')
f= f.groupby('buyer_nm').agg('mean')
f = f.to_dict()['gender']
tr['buyer_nm_target'] = tr.buyer_nm.apply(lambda x: f.get(x,0))
f = tr.groupby('custid')['buyer_nm_target'].agg({'buyer_nm_tar_mean':'mean'}).reset_index()
features.append(f); f.head(2)

In [ ]:
f = tr.groupby('custid')['tot_amt'].agg([('총구매액', 'sum')]).reset_index()
features.append(f); f.head(2)

In [ ]:
f = tr.groupby('custid')['tot_amt'].agg([('구매건수', 'size')]).reset_index()
features.append(f); f.head(2)

In [ ]:
f = tr.groupby('custid')['tot_amt'].agg([('평균구매가격', 'mean')]).reset_index()
features.append(f); f.head(2)

In [ ]:
f = tr.groupby('custid')['inst_mon'].agg([('평균할부개월수', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f); f.head(2)

In [ ]:
tr['sdate'] = tr.sales_date.str[:10]
f = tr.groupby(by = 'custid')['sdate'].agg([('내점일수','nunique')]).reset_index()
features.append(f); f.head(2)

In [ ]:
x = tr[tr['import_flg'] == 1].groupby('custid').size() / tr.groupby('custid').size()
f = x.reset_index().rename(columns={0: '수입상품_구매비율'}).fillna(0)
f.iloc[:,1] = (f.iloc[:,1]*100).apply(round, args=(1,))
features.append(f); f.head(2)

In [ ]:
def f2(x):
    k = x.dayofweek
    if k <= 2 :
        return('월화수_구매건수')
    elif 3 <= k < 5 :
        return('목금_구매건수')
    elif 5 <= k < 6 :
        return('토_구매건수')
    else :
        return('일_구매건수')    
    
tr['요일2'] = pd.to_datetime(tr.sales_date).apply(f2)
f = pd.pivot_table(tr, index='custid', columns='요일2', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f); f.head(2)

In [ ]:
def f1(x):
    k = x.month
    if 2 <= k <= 4 :
        return('234월_구매건수')
    elif 5 <= k <= 7 :
        return('567월_구매건수')
    elif 8 <= k <= 10 :
        return('8910월_구매건수')
    else :
        return('11121월_구매건수')    
    
tr['season2'] = pd.to_datetime(tr.sales_date).apply(f1)
f = pd.pivot_table(tr, index='custid', columns='season2', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f); f.head(2)

In [ ]:
def f2(x):
    if 901 <= x < 1200 :
        return('12시 이전_구매건수')
    elif 1200 <= x < 1400 :
        return('12~2시_구매건수')
    elif 1400 <= x < 1600 :
        return('2~4시_구매건수')
    elif 1600 <= x < 1800 :
        return('4~6시_구매건수')
    else :
        return('6시이후_구매건수')  

tr['timeslot2'] = tr.sales_time.apply(f2)

In [ ]:
f = pd.pivot_table(tr, index='custid', columns='timeslot2', values='tot_amt',
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f); f.head(2)

In [ ]:
f = tr.groupby('custid')['dis_amt'].agg([('평균할인금액', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f); f.head(2)

In [ ]:
f = tr.groupby('custid')['sales_time'].agg([('평균구매시간', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f); f.head(2)

In [ ]:
f = tr.groupby('custid')['net_amt'].agg([('실제구매금액', 'sum')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f); f.head(2)

In [ ]:
f = tr.groupby('custid')['net_amt'].agg([('실제구매금액평균', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f); f.head(2)

In [ ]:
f = pd.pivot_table(tr, index='custid', columns='str_nm', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f); f.head(2)

In [ ]:
tr.loc[456697,"sales_time"] = 1530
tr["time"] = tr['sdate'].astype(str).str.cat(tr["sales_time"].astype(str), sep =" ") 
tr["time"] = pd.to_datetime(tr.time, format='%Y-%m-%d %H%M')

In [ ]:
f = tr.groupby(['sdate','custid'])['time'].agg([('time', ['min','max'])]).reset_index()
f['shopping_time'] = (f['time']['max'] - f['time']['min']).dt.total_seconds()
f.drop(['sdate','time'], axis=1, inplace=True,level=0)
f = f.groupby(['custid'])['shopping_time'].agg([('shopping_time_mean','mean')]).reset_index()
features.append(f); f.head(2)

In [ ]:
f = pd.pivot_table(tr, index='custid', columns='part_nm', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f); f.head(2)

In [ ]:
f = pd.pivot_table(tr, index='custid', columns='buyer_nm', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f); f.head(2)

In [ ]:
X_train = pd.DataFrame({'custid': tr_train.custid.unique()})
for f in features :
    X_train = pd.merge(X_train, f, how='left')
display(X_train)

X_test = pd.DataFrame({'custid': tr_test.custid.unique()})
for f in features :
    X_test = pd.merge(X_test, f, how='left')
display(X_test)

In [ ]:
X_train["평균할인율"] = (X_train["평균할인금액"] / X_train["평균구매가격"])*100
X_test["평균할인율"] = (X_test["평균할인금액"] / X_test["평균구매가격"])*100
X_train["토_비율"] = (X_train["토_구매건수"] / X_train["구매건수"])*100
X_train["일_비율"] = (X_train["일_구매건수"] / X_train["구매건수"])*100
X_test["토_비율"] = (X_test["토_구매건수"] / X_test["구매건수"])*100
X_test["일_비율"] = (X_test["일_구매건수"] / X_test["구매건수"])*100

In [ ]:
X_train['가정용품R']=X_train['가정용품']+X_train['가정용품파트']
X_train['공산품R']=X_train['공산품']+X_train['공산품파트']
X_train['로얄부띠끄R']=X_train['로얄부띠끄']+X_train['로얄부틱']
X_train['생식품R']=X_train['생식품']+X_train['생식품파트']
X_train['스포츠캐주얼R']=X_train['스포츠캐주얼']+X_train['스포츠캐쥬얼']
X_train['여성캐주얼R']=X_train['여성캐주얼']+X_train['여성캐쥬얼']
X_train['잡화R']=X_train['잡화']+X_train['잡화파트']

X_test['가정용품R']=X_test['가정용품']+X_test['가정용품파트']
X_test['공산품R']=X_test['공산품']+X_test['공산품파트']
X_test['로얄부띠끄R']=X_test['로얄부띠끄']+X_test['로얄부틱']
X_test['생식품R']=X_test['생식품']+X_test['생식품파트']
X_test['스포츠캐주얼R']=X_test['스포츠캐주얼']+X_test['스포츠캐쥬얼']
X_test['여성캐주얼R']=X_test['여성캐주얼']+X_test['여성캐쥬얼']
X_test['잡화R']=X_test['잡화']+X_test['잡화파트']

In [ ]:
X_train['남성파트']=X_train['가정용품R']+X_train['공산품R']+X_train['생식품R']+X_train['케주얼,구두,아동']
X_test['남성파트']=X_test['가정용품R']+X_test['공산품R']+X_test['생식품R']+X_test['케주얼,구두,아동']
X_train['여성파트']=X_train['여성캐주얼']+X_train['영캐릭터']+X_train['영플라자']+X_train['패션잡화'] 
X_test['여성파트']=X_test['여성캐주얼']+X_test['영캐릭터']+X_test['영플라자']+X_test['패션잡화'] 

In [ ]:
def f1(x):
    if  x <= 2000000 :
        return('1')
    elif 2000001 <= x < 5000000 :
        return('2')
    else :
        return('3')  

X_train['총구매액B'] = X_train.총구매액.apply(f1)
X_train['총구매액B'] = X_train['총구매액B'].astype(int)

In [ ]:
def f1(x):
    if  x <= 2000000 :
        return('1')
    elif 2000001 <= x < 5000000 :
        return('2')
    else :
        return('3')  

X_test['총구매액B'] = X_test.총구매액.apply(f1)
X_test['총구매액B'] = X_test['총구매액B'].astype(int)

In [ ]:
def f1(x):
    if  x <= 1500 :
        return('4')
    elif 1501 <= x < 1600 :
        return('3')
    elif 1601 <= x < 1700 :
        return('2')
    else :
        return('1')  

X_train['평균구매시간B'] = X_train.평균구매시간.apply(f1)
X_train['평균구매시간B'] = X_train['평균구매시간B'].astype(int)

In [ ]:
def f1(x):
    if  x <= 1500 :
        return('4')
    elif 1501 <= x < 1600 :
        return('3')
    elif 1601 <= x < 1700 :
        return('2')
    else :
        return('1')  

X_test['평균구매시간B'] = X_test.평균구매시간.apply(f1)
X_test['평균구매시간B'] = X_test['평균구매시간B'].astype(int)

In [ ]:
def f1(x):
    if  x <= 2 :
        return('1')
    elif 2.0001 <= x < 3 :
        return('2')
    elif 3.0001 <= x < 4 :
        return('3')
    elif 4.0001 <= x < 5 :
        return('4')
    else :
        return('5')  

X_train['평균할인율B'] = X_train.평균할인율.apply(f1)
X_train['평균할인율B'] = X_train['평균할인율B'].astype(int)

In [ ]:
def f1(x):
    if  x <= 2 :
        return('1')
    elif 2.0001 <= x < 3 :
        return('2')
    elif 3.0001 <= x < 4 :
        return('3')
    elif 4.0001 <= x < 5 :
        return('4')
    else :
        return('5')  

X_test['평균할인율B'] = X_test.평균할인율.apply(f1)
X_test['평균할인율B'] = X_test['평균할인율B'].astype(int)

In [ ]:
def f1(x):
    if  x < 1 :
        return('1')
    elif 1 <= x <= 10 :
        return('2')
    elif 11 <= x <= 20 :
        return('3')
    else :
        return('4')  

X_train['남성파트B'] = X_train.남성파트.apply(f1)
X_train['남성파트B'] = X_train['남성파트B'].astype(int)

In [ ]:
def f1(x):
    if  x < 1 :
        return('1')
    elif 1 <= x <= 10 :
        return('2')
    elif 11 <= x <= 20 :
        return('3')
    else :
        return('4')  

X_test['남성파트B'] = X_test.남성파트.apply(f1)
X_test['남성파트B'] = X_test['남성파트B'].astype(int)

In [ ]:
def f1(x):
    if  x < 1 :
        return('3')
    elif 1 <= x <= 4 :
        return('2')
    else :
        return('1')  

X_train['여성파트B'] = X_train.여성파트.apply(f1)
X_train['여성파트B'] = X_train['여성파트B'].astype(int)

In [ ]:
def f1(x):
    if  x < 1 :
        return('3')
    elif 1 <= x <= 4 :
        return('2')
    else :
        return('1')  

X_test['여성파트B'] = X_test.여성파트.apply(f1)
X_test['여성파트B'] = X_test['여성파트B'].astype(int)


In [ ]:
X_train['구매_시간'] = X_train['총구매액B'] * X_train['평균구매시간B']
X_train['구매_할인'] = X_train['총구매액B'] * X_train['평균할인율B']
X_train['할인_시간'] = X_train['평균구매시간B'] * X_train['평균할인율B']
X_train['구매_할인_시간'] = X_train['평균구매시간B'] * X_train['평균할인율B']* X_train['총구매액B']
X_train['남성_여성'] = X_train['남성파트B'] * X_train['여성파트B']
X_train['구매_남성'] = X_train['총구매액B'] * X_train['남성파트B']
X_train['구매_여성'] = X_train['총구매액B'] * X_train['여성파트B']
X_train['할인_남성'] = X_train['평균할인율B'] * X_train['남성파트B']
X_train['할인_여성'] = X_train['평균할인율B'] * X_train['여성파트B']
X_train['시간_남성'] = X_train['평균구매시간B'] * X_train['남성파트B']
X_train['시간_여성'] = X_train['평균구매시간B'] * X_train['여성파트B']

X_test['구매_시간'] = X_test['총구매액B'] * X_test['평균구매시간B']
X_test['구매_할인'] = X_test['총구매액B'] * X_test['평균할인율B']
X_test['할인_시간'] = X_test['평균구매시간B'] * X_test['평균할인율B']
X_test['구매_할인_시간'] = X_test['평균구매시간B'] * X_test['평균할인율B']* X_test['총구매액B']
X_test['남성_여성'] = X_test['남성파트B'] * X_test['여성파트B']
X_test['구매_남성'] = X_test['총구매액B'] * X_test['남성파트B']
X_test['구매_여성'] = X_test['총구매액B'] * X_test['여성파트B']
X_test['할인_남성'] = X_test['평균할인율B'] * X_test['남성파트B']
X_test['할인_여성'] = X_test['평균할인율B'] * X_test['여성파트B']
X_test['시간_남성'] = X_test['평균구매시간B'] * X_test['남성파트B']
X_test['시간_여성'] = X_test['평균구매시간B'] * X_test['여성파트B']

In [ ]:
f = X_train.총구매액.where(X_train.총구매액>=0, other=0)
f = np.log(f+1)
X_train.총구매액 = f

f = X_test.총구매액.where(X_test.총구매액>=0, other=0)
f = np.log(f+1)
X_test.총구매액 = f

f = X_train.평균구매가격.where(X_train.평균구매가격>=0, other=0)
f = np.log(f+1)
X_train.평균구매가격 = f

f = X_test.평균구매가격.where(X_test.평균구매가격>=0, other=0)
f = np.log(f+1)
X_test.평균구매가격 = f

f = X_train.구매건수.where(X_train.구매건수>=0, other=0)
f = np.log(f+1)
X_train.구매건수 = f

f = X_test.구매건수.where(X_test.구매건수>=0, other=0)
f = np.log(f+1)
X_test.구매건수 = f

f = X_train.내점일수.where(X_train.내점일수>=0, other=0)
f = np.log(f+1)
X_train.내점일수 = f

f = X_test.내점일수.where(X_test.내점일수>=0, other=0)
f = np.log(f+1)
X_test.내점일수 = f


f = X_train.실제구매금액평균.where(X_train.실제구매금액평균>=0, other=0)
f = np.log(f+1)
X_train.실제구매금액평균 = f

f = X_test.실제구매금액평균.where(X_test.실제구매금액평균>=0, other=0)
f = np.log(f+1)
X_test.실제구매금액평균 = f

f = X_train.실제구매금액.where(X_train.실제구매금액>=0, other=0)
f = np.log(f+1)
X_train.실제구매금액 = f

f = X_test.실제구매금액.where(X_test.실제구매금액>=0, other=0)
f = np.log(f+1)
X_test.실제구매금액 = f

In [ ]:
IDtest = X_test.custid;
X_train.drop(['custid'], axis=1, inplace=True)
X_test.drop(['custid'], axis=1, inplace=True)
y_train = pd.read_csv('y_train.csv').gender

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=0, max_depth=6)

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bytree=0.8, gamma=5, learning_rate=0.1, 
                         max_delta_step=0,  max_depth=5, min_child_weight=5, missing=None, n_estimators=100, n_jobs=1, nthread=None,
                         objective='binary:logistic', random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
                         silent=True, subsample=0.6)

In [ ]:
from sklearn.ensemble import VotingClassifier
votingC = VotingClassifier(estimators=[('gbc', gbc),('clf', clf)], voting='soft')
pred = votingC.fit(X_train, y_train).predict_proba(X_test)[:,1]

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=0, max_depth=6)
pred1 = gbc.fit(X_train, y_train).predict_proba(X_test)[:,1]

In [ ]:
from xgboost import XGBClassifier
clf = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bytree=0.8, gamma=5, learning_rate=0.1, 
                         max_delta_step=0,  max_depth=5, min_child_weight=5, missing=None, n_estimators=100, n_jobs=1, nthread=None,
                         objective='binary:logistic', random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
                         silent=True, subsample=0.6)
pred2 = clf.fit(X_train, y_train).predict_proba(X_test)[:,1]

In [ ]:
max_features = X_train.shape[1]
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler  # StandardScaler 등 다른 Scaler로 바꾸면 어떻게 될까?
#scaler = MinMaxScaler()
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from keras import models
from keras import layers
from keras.optimizers import RMSprop
from keras import regularizers
from keras.callbacks import EarlyStopping

# 이 DNN은 4개의 은익층으로 구성되어 있으나, 은익층의 수나 은익층의 노드 수를 변경하면 어떨까?
model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(max_features,), kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
#model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

# batch_size도 성능에 영향을 미친다.
model.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=10, batch_size=128, 
                    validation_split=0.3, callbacks=[EarlyStopping(patience=5)])

In [ ]:
history_dict =history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='train loss')
plt.plot(epochs, val_loss, 'b', label='validation loss')
plt.legend()
plt.title("Traing and validation loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
plt.clf()
acc = history_dict['acc']
val_acc = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='train acc')
plt.plot(epochs, val_acc, 'b', label='validation acc')
plt.legend()
plt.title("Traing and validation acc")
plt.xlabel('Epochs')
plt.ylabel('acc')
plt.show()

In [ ]:
test_loss,test_acc = model.evaluate(X_train, y_train)
print(test_acc)

In [ ]:
pred3 = model.predict(X_test)[:,0]

In [ ]:
tr_train = pd.read_csv('X_train.csv', encoding='cp949')
tr_test = pd.read_csv('X_test.csv', encoding='cp949')
tr = pd.concat([tr_train, tr_test])
tr['real_amt']= tr.tot_amt / tr.inst_mon
tr['sdate'] = tr.sales_date.str[:10]
tr['dist_rate'] = (tr['dis_amt']/tr['tot_amt'])*100
tr.loc[456697,"sales_time"] = 1530
tr["time"] = tr['sdate'].astype(str).str.cat(tr["sales_time"].astype(str), sep =" ") 
tr["time"] = pd.to_datetime(tr.time, format='%Y-%m-%d %H%M')

In [ ]:
features = []

In [ ]:
#평균할인율
f = tr.groupby('custid')['dist_rate'].agg([('dis_rate', 'mean')]).reset_index()
features.append(f)

In [ ]:
#평균시간
f = tr.groupby(['custid'])['sales_time'].agg([('sales_time', 'mean')]).reset_index()
features.append(f)

In [ ]:
#남성파트
df = tr.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df['part_nm'] = np.where(df.part_nm.str.contains('남성'), '남성', '비남성')
df = df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
df['남성part'] = (df['남성'] / (df['남성'] + df['비남성'])) * 100
df = df.fillna(0)
features.append(df)

In [ ]:
#화장품구매비율
df = tr.groupby(['custid','corner_nm'])['tot_amt'].agg([('tot_amt_corner', 'sum')]).reset_index()
df['corner_nm'] = np.where(df.corner_nm.str.contains('화장품'), '화장품', '비화장품')
df = df.pivot_table(values='tot_amt_corner', index=df.custid, columns='corner_nm', aggfunc='first',fill_value=0).reset_index()
df['화장품비율'] = (df['화장품'] / (df['화장품'] + df['비화장품'])) * 100
df = df.fillna(0)
features.append(df)

In [ ]:
#쇼핑시간
df = tr.groupby(['sdate','custid'])['time'].agg([('time', ['min','max'])]).reset_index()
df['shopping_time'] = (df['time']['max'] - df['time']['min']).dt.total_seconds()
df.drop(['sdate','time'], axis=1, inplace=True,level=0)
df = df.groupby(['custid'])['shopping_time'].agg([('shopping_time_mean','mean')]).reset_index()
features.append(df)

In [ ]:
#할부대비평균실구매
f = tr.groupby('custid')['real_amt'].agg([('real_amt', 'mean')]).reset_index()
features.append(f)

In [ ]:
#평균구매상품종류
df =tr.groupby(['custid','goodcd'])['tot_amt'].agg([('good_count', 'count')]).reset_index()
f = df.groupby(['custid'])['good_count'].agg([('good_count_mean', 'mean')]).reset_index()
features.append(f)

In [ ]:
#지역
df = tr.groupby(['custid','str_nm'])['tot_amt'].agg([('tot_amt_str', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_str', index=df.custid, columns='str_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#팀별
df = tr.groupby(['custid','team_nm'])['tot_amt'].agg([('tot_amt_team', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_team', index=df.custid, columns='team_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#총구매수입상품
df = tr.groupby(['custid'])['import_flg'].agg([('import_flg_sum', 'sum')]).reset_index()
features.append(df)

In [ ]:
#월별총구매수입상품
df = tr.groupby(['custid'])['import_flg'].agg([('inst_mon_sum', 'sum')]).reset_index()
features.append(df)

In [ ]:
#파트
df = tr.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#코너
df = tr.groupby(['custid','corner_nm'])['tot_amt'].agg([('tot_amt_corner', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_corner', index=df.custid, columns='corner_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#pc
df = tr.groupby(['custid','pc_nm'])['tot_amt'].agg([('tot_amt_pc', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_pc', index=df.custid, columns='pc_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#브랜드
df = tr.groupby(['custid','brd_nm'])['tot_amt'].agg([('tot_amt_brd', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_brd', index=df.custid, columns='brd_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#구매자
df = tr.groupby(['custid','buyer_nm'])['tot_amt'].agg([('tot_amt_buyer', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_buyer', index=df.custid, columns='buyer_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#구매시간
from datetime import timedelta as dt
test = tr.groupby(['custid'])['sales_date'].agg([('sales_date', 'max')]).reset_index()
test['days'] = (pd.to_datetime('2002-01-01') - pd.to_datetime(test.sales_date)).dt.days
test.drop(['sales_date'], axis=1, inplace=True)
features.append(test)

In [ ]:
#일평균구매액
test2 = tr.groupby(['sales_date','custid'])['tot_amt'].agg([('day_amt', 'sum')]).reset_index()
test2 = test2.groupby(['custid'])['day_amt'].agg([('일평균구매액', 'mean')]).reset_index()
features.append(test2)

In [ ]:
#일평균구매건
df = tr.groupby(['sales_date','custid'])['custid'].agg([('day_visit', 'count')]).reset_index()
f = df.groupby(['custid'])['day_visit'].agg([('일평균구매건', 'mean')]).reset_index()
features.append(f)

In [ ]:
#총구매액
f = tr.groupby('custid')['tot_amt'].agg([('총구매액', 'sum')]).reset_index()
features.append(f)

In [ ]:
f = tr.groupby('custid')['tot_amt'].agg([('구매건수', 'size')]).reset_index()
features.append(f)

In [ ]:
f = tr.groupby('custid')['tot_amt'].agg([('평균구매가격', 'mean')]).reset_index()
features.append(f)

In [ ]:
f = tr.groupby('custid')['inst_mon'].agg([('평균할부개월수', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f)

In [ ]:
n = tr.corner_nm.nunique()
f = tr.groupby('custid')['brd_nm'].agg([('구매상품다양성', lambda x: len(x.unique()) / n)]).reset_index()
features.append(f)

In [ ]:
tr['sdate'] = tr.sales_date.str[:10]
f = tr.groupby(by = 'custid')['sdate'].agg([('내점일수','nunique')]).reset_index()
features.append(f)

In [ ]:
x = tr[tr['import_flg'] == 1].groupby('custid').size() / tr.groupby('custid').size()
f = x.reset_index().rename(columns={0: '수입상품_구매비율'}).fillna(0)
f.iloc[:,1] = (f.iloc[:,1]*100).apply(round, args=(1,))
features.append(f)

In [ ]:
def fw(x):
    k = x.dayofweek
    if k <= 4 :
        return('주중_방문')
    else :
        return('주말_방문')    
    
df = tr.copy()
df = df.drop_duplicates(['custid','sales_date'])

df['week'] = pd.to_datetime(df.sales_date).apply(fw)
df = pd.pivot_table(df, index='custid', columns='week', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
df['주말방문비율'] = ((df.iloc[:,1] / (df.iloc[:,1]+df.iloc[:,2]))*100).apply(round, args=(1,))
f = df.copy().iloc[:,[0,-1]]
features.append(f)

In [ ]:
def f1(x):
    k = x.month
    if 3 <= k <= 5 :
        return('봄-구매건수')
    elif 6 <= k <= 8 :
        return('여름-구매건수')
    elif 9 <= k <= 11 :    
        return('가을-구매건수')
    else :
        return('겨울-구매건수')    
    
tr['season'] = pd.to_datetime(tr.sales_date).apply(f1)
f = pd.pivot_table(tr, index='custid', columns='season', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f)

In [ ]:
def f2(x):
    if 9 <= x <= 12 :
        return('아침_구매건수')
    elif 13 <= x <= 17 :
        return('점심_구매건수')
    else :
        return('저녁_구매건수')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리

tr['timeslot'] = tr.sales_date.str.split(' |:', expand=True).iloc[:,1].astype(int).apply(f2)
f = pd.pivot_table(tr, index='custid', columns='timeslot', values='tot_amt', 
                   aggfunc=np.size).reset_index()
features.append(f)

In [ ]:
f = tr.groupby('custid').agg({
    'tot_amt': [('최대구매액', 'max')],
    'dis_amt': [('dis_sum', 'sum'),('dis_mean', 'mean')],
    'inst_mon': [('최대할부개월수', 'max')],
    'brd_nm': [('구매상품다양성비', lambda x: x.nunique()/x.count())],
    'import_flg': [('수입상품_구매수', 'sum')],
    'sales_date': [('내점비율',lambda x: x.str[:10].nunique()/x.count())],
    'sales_time': [('밤구입비율', lambda x: np.count_nonzero(x.astype(np.int)[(x>1800)|(x<900)])/ x.count())],
    }).reset_index()
features.append(f)

In [ ]:
# 차원축소 매소드 
from sklearn.decomposition import PCA

def dummy_to_pca(tr, column_name:str) :
    max_seq = 300
    max_d = 15
    col_count = tr.groupby(column_name)[column_name].count()
    if len(col_count) > max_seq:
        tops = col_count.sort_values(ascending=False)[0:max_seq].index
        f =tr.loc[tr[column_name].isin(tops)][['custid', column_name]]
    else:
        tops = col_count.index
        f =tr[['custid', column_name]]
    f = pd.get_dummies(f, columns=[column_name])  # This method performs One-hot-encoding
    f = f.groupby('custid').mean()
    if len(tops) < max_d:
        max_d = len(tops)
    pca = PCA(n_components=max_d)
    pca.fit(f)
    cumsum = np.cumsum(pca.explained_variance_ratio_) #분산의 설명량을 누적합
    #print(cumsum)
    num_d = np.argmax(cumsum >= 0.99) + 1 # 분산의 설명량이 99%이상 되는 차원의 수
    if num_d == 1:
        num_d = max_d
    pca = PCA(n_components=num_d)    
    result = pca.fit_transform(f)
    result = pd.DataFrame(result)
    result.columns = [column_name + '_' + str(column) for column in result.columns]
    result.index = f.index
    return result.reset_index()

In [ ]:
f = dummy_to_pca(tr, 'brd_nm'); features.append(f)
f = dummy_to_pca(tr, 'corner_nm'); features.append(f)
f = dummy_to_pca(tr, 'pc_nm'); features.append(f)
f = dummy_to_pca(tr, 'part_nm'); features.append(f)
f = dummy_to_pca(tr, 'buyer_nm'); features.append(f)
f = dummy_to_pca(tr, 'team_nm'); features.append(f)
f = dummy_to_pca(tr, 'goodcd'); features.append(f)
f = dummy_to_pca(tr, 'str_nm'); features.append(f)
tr['month'] = pd.to_datetime(tr['sales_date']).dt.month.astype(str)
f = dummy_to_pca(tr, 'month'); features.append(f)
tr['week'] = pd.to_datetime(tr['sales_date']).dt.dayofweek.astype(str)
f = dummy_to_pca(tr, 'week'); features.append(f)
tr['time'] = np.floor(tr['sales_time']/100).astype(int).astype(str)
f = dummy_to_pca(tr, 'time'); features.append(f)

In [ ]:
X_train = pd.DataFrame({'custid': tr_train.custid.unique()})
for f in features :
    X_train = pd.merge(X_train, f, how='left',on='custid')
display(X_train.shape)

X_test = pd.DataFrame({'custid': tr_test.custid.unique()})
for f in features :
    X_test = pd.merge(X_test, f, how='left',on='custid')
display(X_test.shape)

In [ ]:
X_train['평균내점구매액'] = X_train['총구매액']/X_train['내점일수']
X_train['주중방문비율'] = (100 - X_train['주말방문비율'])
X_train['국내상품_구매비율'] = (100 - X_train['수입상품_구매비율'])
X_train['할부구매가격'] = X_train['평균구매가격'] / X_train['평균할부개월수']
X_train['구매상품다양성'] = X_train['총구매액'] / X_train['구매상품다양성']
X_train['주말방문수'] = (X_train['주말방문비율'] * X_train['내점일수']) / 100
X_train['주말방문수'] = X_train['주말방문수'].astype('int64')
X_train['주중방문수'] = X_train['내점일수'] - X_train['주말방문수']
X_train['주중방문수'] = X_train['주중방문수'].astype('int64')
X_train['내점당편균구매건수'] = X_train['구매건수']/X_train['내점일수']
X_train['주중구매액'] = X_train['총구매액']*(X_train['주중방문비율']/100)
X_train['주말구매액'] = X_train['총구매액'] - X_train['주중구매액']

X_test['평균내점구매액'] = X_test['총구매액']/X_test['내점일수']
X_test['주중방문비율'] = (100 - X_test['주말방문비율'])
X_test['국내상품_구매비율'] = (100 - X_test['수입상품_구매비율'])
X_test['할부구매가격'] = X_test['평균구매가격'] / X_test['평균할부개월수']
X_test['구매상품다양성'] = X_test['총구매액'] / X_test['구매상품다양성']
X_test['주말방문수'] = (X_test['주말방문비율'] * X_test['내점일수']) / 100
X_test['주말방문수'] = X_test['주말방문수'].astype('int64')
X_test['주중방문수'] = X_test['내점일수'] - X_test['주말방문수']
X_test['주중방문수'] = X_test['주중방문수'].astype('int64')
X_test['내점당편균구매건수'] = X_test['구매건수']/X_test['내점일수']
X_test['주중구매액'] = X_test['총구매액']*(X_test['주중방문비율']/100)
X_test['주말구매액'] = X_test['총구매액'] - X_test['주중구매액']

In [ ]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

IDtest = X_test.custid;
X_train.drop(['custid'], axis=1, inplace=True)
X_test.drop(['custid'], axis=1, inplace=True)
y_train = pd.read_csv('y_train.csv').gender

In [ ]:
X_train.columns = np.arange(0,len(X_train.columns))
X_test.columns = np.arange(0,len(X_train.columns))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.ensemble import GradientBoostingClassifier
kfold = StratifiedKFold(n_splits=2)
parameters = {'loss' : "deviance",
              'max_depth': 8,
              'min_samples_leaf': 100,
              'max_features': 0.3 
              } 
GBC = GradientBoostingClassifier(**parameters, random_state=123)

In [ ]:
from xgboost import XGBClassifier
parameters = {'xgb__max_depth': 4, 'xgb__subsample': 0.7,'gamma': 2}
clf = XGBClassifier(**parameters, random_state=123, n_jobs=-1)

In [ ]:
from sklearn.ensemble import VotingClassifier
votingC = VotingClassifier(estimators=[('clf', clf), ('gbc', GBC)], voting='soft', n_jobs=-1)


pred4 = votingC.fit(X_train, y_train).predict_proba(X_test)[:,1]

In [ ]:
tr_train = pd.read_csv('X_train.csv', encoding='cp949')
tr_test = pd.read_csv('X_test.csv', encoding='cp949')
tr = pd.concat([tr_train, tr_test])
tr['real_amt']= tr.tot_amt / tr.inst_mon
tr['sdate'] = tr.sales_date.str[:10]
tr['dist_rate'] = (tr['dis_amt']/tr['tot_amt'])*100
tr.loc[456697,"sales_time"] = 1530
features = []
tr["time"] = tr['sdate'].astype(str).str.cat(tr["sales_time"].astype(str), sep =" ") 
tr["time"] = pd.to_datetime(tr.time, format='%Y-%m-%d %H%M')

In [ ]:
#평균할인율
f = tr.groupby('custid')['dist_rate'].agg([('dis_rate', 'mean')]).reset_index()
features.append(f)

In [ ]:
#평균시간
f = tr.groupby(['custid'])['sales_time'].agg([('sales_time', 'mean')]).reset_index()
features.append(f)

In [ ]:
#남성파트
df = tr.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df['part_nm'] = np.where(df.part_nm.str.contains('남성'), '남성', '비남성')
df = df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
df['남성part'] = (df['남성'] / (df['남성'] + df['비남성'])) * 100
df = df.fillna(0)
features.append(df)

In [ ]:
#화장품구매비율
df = tr.groupby(['custid','corner_nm'])['tot_amt'].agg([('tot_amt_corner', 'sum')]).reset_index()
df['corner_nm'] = np.where(df.corner_nm.str.contains('화장품'), '화장품', '비화장품')
df = df.pivot_table(values='tot_amt_corner', index=df.custid, columns='corner_nm', aggfunc='first',fill_value=0).reset_index()
df['화장품비율'] = (df['화장품'] / (df['화장품'] + df['비화장품'])) * 100
df = df.fillna(0)
features.append(df)

In [ ]:
#쇼핑시간
df = tr.groupby(['sdate','custid'])['time'].agg([('time', ['min','max'])]).reset_index()
df['shopping_time'] = (df['time']['max'] - df['time']['min']).dt.total_seconds()
df.drop(['sdate','time'], axis=1, inplace=True,level=0)
df = df.groupby(['custid'])['shopping_time'].agg([('shopping_time_mean','mean')]).reset_index()
features.append(df)

In [ ]:
#할부대비평균실구매
f = tr.groupby('custid')['real_amt'].agg([('real_amt', 'mean')]).reset_index()
features.append(f)

In [ ]:
#평균구매상품종류
df =tr.groupby(['custid','goodcd'])['tot_amt'].agg([('good_count', 'count')]).reset_index()
f = df.groupby(['custid'])['good_count'].agg([('good_count_mean', 'mean')]).reset_index()
features.append(f)

In [ ]:
#지역
df = tr.groupby(['custid','str_nm'])['tot_amt'].agg([('tot_amt_str', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_str', index=df.custid, columns='str_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#팀별
df = tr.groupby(['custid','team_nm'])['tot_amt'].agg([('tot_amt_team', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_team', index=df.custid, columns='team_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#총구매수입상품
df = tr.groupby(['custid'])['import_flg'].agg([('import_flg_sum', 'sum')]).reset_index()
features.append(df)

In [ ]:
#월별총구매수입상품
df = tr.groupby(['custid'])['import_flg'].agg([('inst_mon_sum', 'sum')]).reset_index()
features.append(df)

In [ ]:
#파트
df = tr.groupby(['custid','part_nm'])['tot_amt'].agg([('tot_amt_part', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_part', index=df.custid, columns='part_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#코너
df = tr.groupby(['custid','corner_nm'])['tot_amt'].agg([('tot_amt_corner', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_corner', index=df.custid, columns='corner_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#pc
df = tr.groupby(['custid','pc_nm'])['tot_amt'].agg([('tot_amt_pc', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_pc', index=df.custid, columns='pc_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#브랜드
df = tr.groupby(['custid','brd_nm'])['tot_amt'].agg([('tot_amt_brd', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_brd', index=df.custid, columns='brd_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#구매자
df = tr.groupby(['custid','buyer_nm'])['tot_amt'].agg([('tot_amt_buyer', 'sum')]).reset_index()
df =df.pivot_table(values='tot_amt_buyer', index=df.custid, columns='buyer_nm', aggfunc='first',fill_value=0).reset_index()
features.append(df)

In [ ]:
#구매시간
from datetime import timedelta as dt
test = tr.groupby(['custid'])['sales_date'].agg([('sales_date', 'max')]).reset_index()
test['days'] = (pd.to_datetime('2002-01-01') - pd.to_datetime(test.sales_date)).dt.days
test.drop(['sales_date'], axis=1, inplace=True)
features.append(test)

In [ ]:
f = tr.groupby('custid')['sales_time'].agg([('평균구매시간', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f); f

In [ ]:
f = tr.groupby('custid')['dis_amt'].agg([('평균할인금액', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f); f

In [ ]:
#일평균구매액
test2 = tr.groupby(['sales_date','custid'])['tot_amt'].agg([('day_amt', 'sum')]).reset_index()
test2 = test2.groupby(['custid'])['day_amt'].agg([('일평균구매액', 'mean')]).reset_index()
features.append(test2)

In [ ]:
#일평균구매건
df = tr.groupby(['sales_date','custid'])['custid'].agg([('day_visit', 'count')]).reset_index()
f = df.groupby(['custid'])['day_visit'].agg([('일평균구매건', 'mean')]).reset_index()
features.append(f)

In [ ]:
#총구매액
f = tr.groupby('custid')['tot_amt'].agg([('총구매액', 'sum')]).reset_index()
features.append(f)

In [ ]:
f = tr.groupby('custid')['tot_amt'].agg([('구매건수', 'size')]).reset_index()
features.append(f)

In [ ]:
f = tr.groupby('custid')['tot_amt'].agg([('평균구매가격', 'mean')]).reset_index()
features.append(f)

In [ ]:
f = tr.groupby('custid')['inst_mon'].agg([('평균할부개월수', 'mean')]).reset_index()
f.iloc[:,1] = f.iloc[:,1].apply(round, args=(1,))
features.append(f)

In [ ]:
n = tr.corner_nm.nunique()
f = tr.groupby('custid')['brd_nm'].agg([('구매상품다양성', lambda x: len(x.unique()) / n)]).reset_index()
features.append(f)

In [ ]:
tr['sdate'] = tr.sales_date.str[:10]
f = tr.groupby(by = 'custid')['sdate'].agg([('내점일수','nunique')]).reset_index()
features.append(f)

In [ ]:
x = tr[tr['import_flg'] == 1].groupby('custid').size() / tr.groupby('custid').size()
f = x.reset_index().rename(columns={0: '수입상품_구매비율'}).fillna(0)
f.iloc[:,1] = (f.iloc[:,1]*100).apply(round, args=(1,))
features.append(f)

In [ ]:
def fw(x):
    k = x.dayofweek
    if k <= 4 :
        return('주중_방문')
    else :
        return('주말_방문')    
    
df = tr.copy()
df = df.drop_duplicates(['custid','sales_date'])

df['week'] = pd.to_datetime(df.sales_date).apply(fw)
df = pd.pivot_table(df, index='custid', columns='week', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
df['주말방문비율'] = ((df.iloc[:,1] / (df.iloc[:,1]+df.iloc[:,2]))*100).apply(round, args=(1,))
f = df.copy().iloc[:,[0,-1]]
features.append(f)

In [ ]:
def f1(x):
    k = x.month
    if 3 <= k <= 5 :
        return('봄-구매건수')
    elif 6 <= k <= 8 :
        return('여름-구매건수')
    elif 9 <= k <= 11 :    
        return('가을-구매건수')
    else :
        return('겨울-구매건수')    
    
tr['season'] = pd.to_datetime(tr.sales_date).apply(f1)
f = pd.pivot_table(tr, index='custid', columns='season', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features.append(f)

In [ ]:
def f2(x):
    if 9 <= x <= 12 :
        return('아침_구매건수')
    elif 13 <= x <= 17 :
        return('점심_구매건수')
    else :
        return('저녁_구매건수')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리

tr['timeslot'] = tr.sales_date.str.split(' |:', expand=True).iloc[:,1].astype(int).apply(f2)
f = pd.pivot_table(tr, index='custid', columns='timeslot', values='tot_amt', 
                   aggfunc=np.size).reset_index()
features.append(f)

In [ ]:
f = tr.groupby('custid')['corner_nm'].agg([('주구매코너', lambda x: x.value_counts().index[0])]).reset_index()
f = pd.get_dummies(f, columns=['주구매코너'])  # This method performs One-hot-encoding
features.append(f)

In [ ]:
X_train = pd.DataFrame({'custid': tr_train.custid.unique()})
for f in features :
    X_train = pd.merge(X_train, f, how='left',on='custid')
display(X_train.shape)

X_test = pd.DataFrame({'custid': tr_test.custid.unique()})
for f in features :
    X_test = pd.merge(X_test, f, how='left',on='custid')

In [ ]:
X_train['평균내점구매액'] = X_train['총구매액']/X_train['내점일수']
X_train['주중방문비율'] = (100 - X_train['주말방문비율'])
X_train['국내상품_구매비율'] = (100 - X_train['수입상품_구매비율'])
X_train['할부구매가격'] = X_train['평균구매가격'] / X_train['평균할부개월수']
X_train['구매상품다양성'] = X_train['총구매액'] / X_train['구매상품다양성']
X_train['주말방문수'] = (X_train['주말방문비율'] * X_train['내점일수']) / 100
X_train['주말방문수'] = X_train['주말방문수'].astype('int64')
X_train['주중방문수'] = X_train['내점일수'] - X_train['주말방문수']
X_train['주중방문수'] = X_train['주중방문수'].astype('int64')
X_train['내점당편균구매건수'] = X_train['구매건수']/X_train['내점일수']
X_train['주중구매액'] = X_train['총구매액']*(X_train['주중방문비율']/100)
X_train['주말구매액'] = X_train['총구매액'] - X_train['주중구매액']

In [ ]:
X_test['평균내점구매액'] = X_test['총구매액']/X_test['내점일수']
X_test['주중방문비율'] = (100 - X_test['주말방문비율'])
X_test['국내상품_구매비율'] = (100 - X_test['수입상품_구매비율'])
X_test['할부구매가격'] = X_test['평균구매가격'] / X_test['평균할부개월수']
X_test['구매상품다양성'] = X_test['총구매액'] / X_test['구매상품다양성']
X_test['주말방문수'] = (X_test['주말방문비율'] * X_test['내점일수']) / 100
X_test['주말방문수'] = X_test['주말방문수'].astype('int64')
X_test['주중방문수'] = X_test['내점일수'] - X_test['주말방문수']
X_test['주중방문수'] = X_test['주중방문수'].astype('int64')
X_test['내점당편균구매건수'] = X_test['구매건수']/X_test['내점일수']
X_test['주중구매액'] = X_test['총구매액']*(X_test['주중방문비율']/100)
X_test['주말구매액'] = X_test['총구매액'] - X_test['주중구매액']

In [ ]:
X_train["평균할인율"] = (X_train["평균할인금액"] / X_train["평균구매가격"])*100
X_test["평균할인율"] = (X_test["평균할인금액"] / X_test["평균구매가격"])*100

In [ ]:
def f1(x):
    if  x <= 2000000 :
        return('1')
    elif 2000001 <= x < 5000000 :
        return('2')
    else :
        return('3')  

X_train['총구매액B'] = X_train.총구매액.apply(f1)
X_train['총구매액B'] = X_train['총구매액B'].astype(int)

In [ ]:
def f1(x):
    if  x <= 2000000 :
        return('1')
    elif 2000001 <= x < 5000000 :
        return('2')
    else :
        return('3')  

X_test['총구매액B'] = X_test.총구매액.apply(f1)
X_test['총구매액B'] = X_test['총구매액B'].astype(int)

In [ ]:
def f1(x):
    if  x <= 1500 :
        return('4')
    elif 1501 <= x < 1600 :
        return('3')
    elif 1601 <= x < 1700 :
        return('2')
    else :
        return('1')  

X_train['평균구매시간B'] = X_train.평균구매시간.apply(f1)
X_train['평균구매시간B'] = X_train['평균구매시간B'].astype(int)

In [ ]:
def f1(x):
    if  x <= 1500 :
        return('4')
    elif 1501 <= x < 1600 :
        return('3')
    elif 1601 <= x < 1700 :
        return('2')
    else :
        return('1')  

X_test['평균구매시간B'] = X_test.평균구매시간.apply(f1)
X_test['평균구매시간B'] = X_test['평균구매시간B'].astype(int)

In [ ]:
def f1(x):
    if  x <= 2 :
        return('1')
    elif 2.0001 <= x < 3 :
        return('2')
    elif 3.0001 <= x < 4 :
        return('3')
    elif 4.0001 <= x < 5 :
        return('4')
    else :
        return('5')  

X_train['평균할인율B'] = X_train.평균할인율.apply(f1)
X_train['평균할인율B'] = X_train['평균할인율B'].astype(int)

In [ ]:
def f1(x):
    if  x <= 2 :
        return('1')
    elif 2.0001 <= x < 3 :
        return('2')
    elif 3.0001 <= x < 4 :
        return('3')
    elif 4.0001 <= x < 5 :
        return('4')
    else :
        return('5')  

X_test['평균할인율B'] = X_test.평균할인율.apply(f1)
X_test['평균할인율B'] = X_test['평균할인율B'].astype(int)

In [ ]:
X_train['구매_시간'] = X_train['총구매액B'] * X_train['평균구매시간B']
X_train['구매_할인'] = X_train['총구매액B'] * X_train['평균할인율B']
X_train['할인_시간'] = X_train['평균구매시간B'] * X_train['평균할인율B']
X_test['구매_시간'] = X_test['총구매액B'] * X_test['평균구매시간B']
X_test['구매_할인'] = X_test['총구매액B'] * X_test['평균할인율B']
X_test['할인_시간'] = X_test['평균구매시간B'] * X_test['평균할인율B']
IDtest = X_test.custid;
X_train.drop(['custid'], axis=1, inplace=True)
X_test.drop(['custid'], axis=1, inplace=True)
y_train = pd.read_csv('y_train.csv').gender
X_train.clip(lower=0,inplace=True)

In [ ]:
X_train.columns = np.arange(0,len(X_train.columns))
X_test.columns = np.arange(0,len(X_train.columns))
max_features = X_train.shape[1]

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
kfold = StratifiedKFold(n_splits=2)
parameters = {'loss' : "deviance",
              'max_depth': 8,
              'min_samples_leaf': 100,
              'max_features': 0.3 
              } 
GBC = GradientBoostingClassifier(**parameters, random_state=123)

In [ ]:
from xgboost import XGBClassifier
parameters = {'xgb__max_depth': 4, 'xgb__subsample': 0.7,'gamma': 2}
clf = XGBClassifier(**parameters, random_state=123, n_jobs=-1)

In [ ]:
from sklearn.ensemble import VotingClassifier
votingC = VotingClassifier(estimators=[('clf', clf), ('gbc', GBC)], voting='soft', n_jobs=-1)
votingC = votingC.fit(X_train, y_train)
pred5 = votingC.fit(X_train, y_train).predict_proba(X_test)[:,1]

In [ ]:
df = tr.groupby(['custid'])['sales_time'].agg([('sales_time', 'mean')]).reset_index()

In [ ]:
def f4(x):
    if x <= 978 :
        return('st_cat01')
    elif 979 <= x <= 1464 :
        return('st_cat02')
    elif 1465 <= x <= 1564 :
        return('st_cat03')
    elif 1565 <= x <= 1658 :
        return('st_cat04')
    else :
        return('st_cat05')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리
df['sale_ct'] = df.sales_time.apply(f4)
df.drop(['sales_time'], axis=1, inplace=True)
df = pd.get_dummies(df, columns=['sale_ct'])
train_1 = df.query('custid not in @IDtest').drop(columns=['custid'])
test_1 = df.query('custid in @IDtest').drop(columns=['custid'])

In [ ]:
df = tr.groupby(['custid'])['dist_rate'].agg([('dist_amt', 'mean')]).reset_index()

In [ ]:
def f3(x):
    if x <= 0 :
        return('cat01')
    elif 1 <= x <= 1.83 :
        return('cat02')
    elif 1.84 <= x <= 2.83 :
        return('cat03')
    elif 2.84 <= x <= 3.92 :
        return('cat04')
    else :
        return('cat05')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리
df['dist'] = df.dist_amt.apply(f3)
df.drop(['dist_amt'], axis=1, inplace=True)
df = pd.get_dummies(df, columns=['dist'])

In [ ]:
train_2 = df.query('custid not in @IDtest').drop(columns=['custid'])
test_2 = df.query('custid in @IDtest').drop(columns=['custid'])

In [ ]:
def f2(x):
    k = x.hour
    if 9 <= k <= 12 :
        return('아침_구매건수')
    elif 13 <= k <= 17 :
        return('점심_구매건수')
    else :
        return('저녁_구매건수')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리

tr['timeslot'] = tr.time.apply(f2)

In [ ]:
def f1(x):
    k = x.month
    if 3 <= k <= 5 :
        return('봄-구매건수')
    elif 6 <= k <= 8 :
        return('여름-구매건수')
    elif 9 <= k <= 11 :    
        return('가을-구매건수')
    else :
        return('겨울-구매건수')    
    
tr['season'] = tr.time.apply(f1)

In [ ]:
def fw(x):
    k = x.dayofweek
    if k <= 4 :
        return('주중_방문')
    else :
        return('주말_방문')    
    
tr['week'] = tr.time.apply(fw)

In [ ]:
tr['sales_hour'] = tr['sales_time']//100;
tr['sales_wkday'] = pd.to_datetime(tr.sales_date).dt.weekday

def makeBOW(col):
    
    f = lambda x: np.where(len(x) >=1, 1, 0)

    train = pd.pivot_table(tr, index='custid', columns=col, values='tot_amt',
                             aggfunc=f, fill_value=0).reset_index(). \
                             query('custid not in @IDtest').drop(columns=['custid'])
    test = pd.pivot_table(tr, index='custid', columns=col, values='tot_amt',
                             aggfunc=f, fill_value=0).reset_index(). \
                             query('custid in @IDtest').drop(columns=['custid'])
    return train, test
train1, test1 = makeBOW('brd_nm')
train2, test2 = makeBOW('corner_nm')
train3, test3 = makeBOW('sales_hour')
train4, test4 = makeBOW('sales_wkday')
train5, test5 = makeBOW('timeslot')
train6, test6 = makeBOW('week')
train7, test7 = makeBOW('season')
#train6, test6 = makeBOW('dis_rate_ca')
X_train = pd.concat([train1, train2, train3,train4,train5,train6,train7,train_1,train_2], axis=1).values
X_test = pd.concat([test1, test2, test3,test4,test5,test6,test7,test_1,test_2], axis=1).values
max_features = X_train.shape[1]

In [ ]:
from sklearn.preprocessing import MinMaxScaler  # StandardScaler 등 다른 Scaler로 바꾸면 어떻게 될까?
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
#scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from keras import models
from keras import layers
from keras.optimizers import RMSprop
from keras import regularizers
from keras.callbacks import EarlyStopping

model3 = models.Sequential()
model3.add(layers.Dense(128, activation='relu', input_shape=(max_features,), kernel_regularizer=regularizers.l2(0.01)))
model3.add(layers.Dense(64, activation='relu'))
model3.add(layers.Dense(32, activation='relu'))
model3.add(layers.Dense(16, activation='relu'))
model3.add(layers.Dense(1, activation='sigmoid'))

model3.summary()

# batch_size도 성능에 영향을 미친다.
model3.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['acc'])
history = model3.fit(X_train, y_train, epochs=8, batch_size=128, 
                    validation_split=0.2, callbacks=[EarlyStopping(patience=5)])

In [ ]:
history_dict =history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='train loss')
plt.plot(epochs, val_loss, 'b', label='validation loss')
plt.legend()
plt.title("Traing and validation loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
plt.clf()
acc = history_dict['acc']
val_acc = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='train acc')
plt.plot(epochs, val_acc, 'b', label='validation acc')
plt.legend()
plt.title("Traing and validation acc")
plt.xlabel('Epochs')
plt.ylabel('acc')
plt.show()

In [ ]:
test_loss,test_acc = model3.evaluate(X_train, y_train)

In [ ]:
pred6 = model3.predict(X_test)[:,0]

In [ ]:
from xgboost import XGBClassifier
parameters = {'n_estimators': 300, 'learning_rate': 0.1}
model = XGBClassifier(**parameters, random_state=123,  n_jobs=-1)
model.fit(X_train, y_train)
pred7 = model.predict_proba(X_test)[:,1]

In [ ]:
max_features = X_train.shape[1]

from sklearn.preprocessing import MinMaxScaler  # StandardScaler 등 다른 Scaler로 바꾸면 어떻게 될까?
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from keras import models
from keras import layers
from keras.optimizers import RMSprop
from keras import regularizers
from keras.callbacks import EarlyStopping

# 이 DNN은 4개의 은익층으로 구성되어 있으나, 은익층의 수나 은익층의 노드 수를 변경하면 어떨까?
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(max_features,), kernel_regularizer=regularizers.l2(0.01)))
#model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01)))

model.summary()

# batch_size도 성능에 영향을 미친다.
model.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=8, batch_size=64, 
                    validation_split=0.2, callbacks=[EarlyStopping(patience=5)])

In [ ]:
history_dict =history.history
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='train loss')
plt.plot(epochs, val_loss, 'b', label='validation loss')
plt.legend()
plt.title("Traing and validation loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
plt.clf()
acc = history_dict['acc']
val_acc = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='train acc')
plt.plot(epochs, val_acc, 'b', label='validation acc')
plt.legend()
plt.title("Traing and validation acc")
plt.xlabel('Epochs')
plt.ylabel('acc')
plt.show()

In [ ]:
test_loss,test_acc = model.evaluate(X_train, y_train)
pred8 = model.predict(X_test)[:,0]
print(test_acc)

In [ ]:
pred_d = (pred+pred1+pred2+pred3+pred4+pred5+pred6+pred7+pred8) / 9

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
tr_train = pd.read_csv('X_train.csv', encoding='cp949')
tr_test = pd.read_csv('X_test.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv', encoding='cp949')
# floor_info = pd.read_csv('floor-info/floor_info.csv', encoding='cp949')
train = pd.merge(tr_train, y_train, how='left')
tr = pd.concat([tr_train, tr_test])
features = {}

In [ ]:
f = tr.groupby('custid')['tot_amt'].agg([('구매건수', 'size')]).reset_index()
features['구매건수'] = f

In [ ]:
f = tr.groupby('custid')['tot_amt'].agg([('평균구매액', 'mean')]).reset_index()
features['평균구매액'] = f

In [ ]:
n = tr.corner_nm.nunique()
f = tr.groupby('custid')['brd_nm'].agg([('구매브랜드다양성', lambda x: len(x.unique()) / n)]).reset_index()
features['구매브랜드다양성'] = f

In [ ]:
tr['sdate'] = tr.sales_date.str[:10]
f = tr.groupby(by = 'custid')['sdate'].agg([('내점일수','nunique')]).reset_index()
features['내점일수'] = f

In [ ]:
x = tr[tr['import_flg'] == 1].groupby('custid').size() / tr.groupby('custid').size()
f = x.reset_index().rename(columns={0: '수입상품_구매비율'}).fillna(0)
f.iloc[:,1] = (f.iloc[:,1]*100).apply(round, args=(1,))
features['수입상품구매비율'] = f

In [ ]:
def wt(x) :
    w = pd.to_datetime(x.sales_date[:10]).dayofweek
    w = { 0:"MON_", 1:"TUE_", 2:"WEN_", 3:"THU_", 4:"FRI_", 5:"SAT_", 6:"SUN_" }.get(w, 6)
    t = np.int(str(x.sales_time)[:2])
    
    if 9 <= t <= 12 :
        t = 'MORNING_구매건수'
    elif 13 <= t <= 17 :
        t = 'LUNCH_구매건수'
    else :
        t = 'DINNER_구매건수'
    return w + t
tr['week_time'] = tr.apply(wt, axis=1)
f = tr[['custid','week_time']]
f.columns = ["custid", "week_time"]
f= f.groupby('custid')['week_time'].value_counts().unstack(level=1).fillna(0).astype(int).reset_index()
features['요일-아침점심저녁별구매건수'] =f;

In [ ]:
def wt2(x) :
    w = pd.to_datetime(x.sales_date[:10]).dayofweek
    w = { 0:"MON_", 1:"TUE_", 2:"WEN_", 3:"THU_", 4:"FRI_", 5:"SAT_", 6:"SUN_" }.get(w, 6)
    t = str(x.sales_time)[:2]
    return w + t
tr['week_time'] = tr.apply(wt2, axis=1)
f = tr[['custid','week_time']]
f.columns = ["custid", "week_time"]
f = f.groupby('custid')['week_time'].value_counts().unstack(level=1).fillna(0).astype(int).reset_index()
features['요일-시간대별구매건수'] =f;

In [ ]:
tr['timeslot'] = tr.sales_time.map(lambda x : "{}시_구매건수".format(str(x)[:2]))
f = pd.pivot_table(tr, index='custid', columns='timeslot', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features['시간대별구매건수'] = f;
tr['timeslot'] = tr.sales_time.map(lambda x : "{}_구매건수".format(str(x)))
f = pd.pivot_table(tr, index='custid', columns='timeslot', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features['분단위구매건수'] = f;
def f1(x):
    if 9 <= x <= 11 :
        return('911_구매건수')
    elif 11 <= x <= 13 :
        return('1113_구매건수')
    elif 13 <= x <= 17 :
        return('1317_구매건수')
    elif 17 <= x <= 22 :
        return('1722_구매건수')
    else :
        return('기타_구매건수')  
    
tr['sales_hour'] = tr.sales_time.map(lambda x : np.int(str(x)[:2]))
tr['timeslot'] = tr['sales_hour'].apply(f1)
f = pd.pivot_table(tr, index='custid', columns='timeslot', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features['5단위시간대별구매건수'] = f.head(2)
def f2(x):
    if 9 <= x <= 12 :
        return('아침_구매건수')
    elif 13 <= x <= 17 :
        return('점심_구매건수')
    else :
        return('저녁_구매건수')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리
    
tr['sales_hour'] = tr.sales_time.map(lambda x : np.int(str(x)[:2]))
tr['timeslot'] = tr['sales_hour'].apply(f2)
f = pd.pivot_table(tr, index='custid', columns='timeslot', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features['아침점심저녁구매건수'] = f

In [ ]:
def f1(x):
    m = x.month
    if x.day<=10 :
        h = "초순"
    elif x.day<=20 :
        h = "중순"
    else :
        h = "하순"  
    return "{}월{}_구매건수".format(m,h)

tr['season'] = pd.to_datetime(tr.sales_date).apply(f1)
f = pd.pivot_table(tr, index='custid', columns='season', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features['세분화계절별구매건수'] = f

In [ ]:
def f2(x):
    if 9 <= x <= 12 :
        return('아침_구매건수')
    elif 13 <= x <= 17 :
        return('점심_구매건수')
    else :
        return('저녁_구매건수')  # datatime 필드가 시간 형식에 맞지 않은 값을 갖는 경우 저녁시간으로 처리
    
tr['sales_hour'] = tr.sales_time.map(lambda x : np.int(str(x)[:2]))
tr['timeslot'] = tr['sales_hour'].apply(f2)
f = pd.pivot_table(tr, index='custid', columns='timeslot', values='tot_amt', 
                   aggfunc=np.size).fillna(0).astype(int).reset_index()
features['아침점심저녁구매건수'] = f;

In [ ]:
def time_transfer(x) :
    hour = np.int(str(x)[:2])
    time = hour * 60
    minute = x%100
    time = time + minute
    return time

tr['sale_time_trans'] = tr.sales_time.map(time_transfer)

tr['sdate'] = tr.sales_date.str[:10]
f = tr.groupby(['custid','sdate'])['sale_time_trans'].agg([('입점시간','min')]).reset_index()
f1 = f
f = tr.groupby(['custid','sdate'])['sale_time_trans'].agg([('퇴점시간','max')]).reset_index()
f2 = f
f = pd.merge(f1,f2,how='left')
f['체류시간'] = f.퇴점시간 - f.입점시간 + 30 #처음구매시점보다 15분전, 마지막구매시점보다 15분후 퇴점이라고 가정함.
f = f.loc[f.체류시간!=0]
f=f.groupby('custid')['체류시간'].agg({"평균체류시간":'mean'}).reset_index()
custid_col = tr[['custid']].drop_duplicates().reset_index(drop=True)
f = pd.merge(custid_col, f, how='left')
f = f.fillna(30).astype(int)
features['예상체류시간'] = f;

In [ ]:
tr['sdate'] = tr.sales_date.str[:10]
f = tr.groupby(['custid','sdate'])['tot_amt'].agg({"구매제품수":'count'}).reset_index().drop("sdate", axis=1)
f = f.groupby("custid")['구매제품수'].agg({"방문당평균구매제품수":'mean'}).reset_index()
features["방문당평균구매제품수"] = f;

In [ ]:
tr['sdate'] = tr.sales_date.str[:10]
def visit_period(x) :
    max_date = pd.to_datetime(x.max()).to_julian_date()
    min_date = pd.to_datetime(x.min()).to_julian_date()
    return max_date - min_date
f = tr.groupby('custid')['sdate'].agg({"총방문기간":visit_period})
features["총방문기간"] = f;

In [ ]:
tr['sdate'] = tr.sales_date.str[:10]
def visit_cycle(x) :
    max_date = pd.to_datetime(x.max()).to_julian_date()
    min_date = pd.to_datetime(x.min()).to_julian_date()
    visit_count = x.count()
    return (max_date - min_date)/visit_count
f = tr.groupby('custid')['sdate'].agg({'평균방문주기':visit_cycle})
features["평균방문주기"] = f;

In [ ]:
f = tr.pivot_table("tot_amt","custid","corner_nm", aggfunc="count").fillna(0).astype(int).reset_index()
features['구매코너별_구매건수'] = f;

In [ ]:
g = train.pivot_table('tot_amt','brd_nm','gender', aggfunc="count").fillna(0).astype(int)
g['women_rate'] = g[0] / g.sum(axis=1)
g = g.sort_values(ascending=False, by='women_rate')
women_brd_list = g[g.sum(axis=1)>=15].head(694).index
tr['women_brd_score'] = tr[['brd_nm']].isin(women_brd_list).astype(int)
f = tr.groupby('custid')[['women_brd_score']].mean().reset_index()
features['여성브랜드구매점수'] = f;

In [ ]:
def make_data(features) :
    X_train = pd.DataFrame({'custid': tr_train.custid.unique()})
    X_test = pd.DataFrame({'custid': tr_test.custid.unique()})
    for f in features.values() :
        X_train = pd.merge(X_train, f, on='custid', how='left')
        X_test = pd.merge(X_test, f, on='custid', how='left')   
    IDtest = X_test.custid;
    X_train.drop(['custid'], axis=1, inplace=True)
    X_test.drop(['custid'], axis=1, inplace=True)
    for idx, val in enumerate(features.keys()) :
        print("적용된 Feature Method{} : {}".format(idx, val))
    return X_train, X_test
brand_gender_index = train.groupby("brd_nm")["gender"].mean().sort_values(ascending=False)
male_brd_list = brand_gender_index.loc[brand_gender_index>=0.9].index.tolist()
female_brd_list = brand_gender_index.loc[brand_gender_index<0.4].index.tolist()
tr['male_brd'] = tr.brd_nm.map(lambda x : 1 if x in male_brd_list else 0)
tr['female_brd'] = tr.brd_nm.map(lambda x : 1 if x in female_brd_list else 0)
f = tr.groupby('custid')['male_brd','female_brd'].mean().reset_index()
features['고성별연관제품구매여부'] = f;
X_train, X_test = make_data(features)
xgb = XGBClassifier(random_state=0)
score = cross_val_score(xgb, X_train, y_train.gender, cv=kfold, scoring="roc_auc", n_jobs=3)
print('{}\nmean = {:.5f}\nstd = {:.5f}'.format(score, score.mean(), score.std()))

In [ ]:
f = tr.pivot_table("tot_amt","custid","brd_nm", aggfunc="count").fillna(0).astype(int).reset_index()
features['브랜드별_구매건수'] = f;

In [ ]:
goodcd = tr.goodcd.value_counts()
tr_good_over_50 = tr[tr.goodcd.isin(goodcd[(goodcd >=250)].index)]
pivot = tr_good_over_50.pivot_table("tot_amt","custid","goodcd", aggfunc="count").fillna(0).astype(int).reset_index()
f = pd.DataFrame({'custid': tr.custid.unique()})
f = f.merge(pivot, on='custid', how='left')
f = f.fillna(0).astype(int)
features['250개이상판매제품_종별구매건수'] = f;

In [ ]:
g = train.pivot_table('tot_amt','goodcd','gender', aggfunc="count").fillna(0).astype(int)
g['women_rate'] = g[0] / g.sum(axis=1)
g = g.sort_values(ascending=False, by='women_rate')
women_brd_list = g[g.sum(axis=1)>=18].head(2408).index
tr['women_brd_score'] = tr[['goodcd']].isin(women_brd_list).astype(int)
f = tr.groupby('custid')[['women_brd_score']].mean().reset_index()
features['여성제품구매점수'] = f;

In [ ]:
f = tr.groupby('custid')['goodcd'].agg([('구매상품다양성', lambda x: len(x.unique()))]).reset_index()
f.구매상품다양성 = np.log(f.구매상품다양성)
features['구매상품다양성'] = f;

In [ ]:
f = tr.groupby('custid')['str_nm'].agg([('방문매장다양성', lambda x: len(x.unique()))]).reset_index()
features['방문매장다양성'] = f;

In [ ]:
f = tr.pivot_table(index='custid', columns='pc_nm',values='brd_nm',aggfunc='count').fillna(0).astype(int)
f.columns = "pc_" + f.columns + "_구매건수"
f.reset_index(inplace=True)
features['pc_nm별구매건수'] = f;

In [ ]:
f = tr.pivot_table(index='custid', columns='part_nm',values='brd_nm',aggfunc='count').fillna(0).astype(int)
f.columns = "part_" + f.columns + "_구매건수"
f.reset_index(inplace=True)
features['part_nm별구매수'] = f;

In [ ]:
f = tr.pivot_table(index='custid', columns='team_nm',values='brd_nm',aggfunc='count').fillna(0).astype(int)
f.columns = "team_" + f.columns + "_구매건수"
f.reset_index(inplace=True)
features['team_nm별구매건수'] = f;

In [ ]:
f = tr.pivot_table(index='custid', columns='buyer_nm',values='brd_nm',aggfunc='count').fillna(0).astype(int)
f.columns = "buyer_" + f.columns + "_구매건수"
f.reset_index(inplace=True)
features['buyer_nm별구매수'] = f;

In [ ]:
# eda = tr.copy()
# eda['sdate'] = eda.sales_date.str[:10]
# eda = pd.merge(left=eda, right=floor_info, how='left', on=['str_nm', 'corner_nm', 'pc_nm'])
# #지점별 층별 정보 스케일링
# def floor_transfer_a(x) :
#     return  {
#         "무역점" : 0,
#         "본점" : 1,
#         "신촌점" : 0,
#         "천호점" : 1
#     }.get(x)

# def floor_transfer_m(x) :  
#     return  {
#         "무역점" : 1/10*13,
#         "본점" : 1/6*13,
#         "신촌점" : 1/11*13,
#         "천호점" : 1
#     }.get(x)
# eda.floor = (eda.floor+eda.str_nm.apply(floor_transfer_a))*eda.str_nm.apply(floor_transfer_m).astype(int)
# eda_unstack = eda.groupby(['custid','sdate','sales_time'])['floor'].value_counts().unstack(level=3)
# eda_unstack = eda_unstack.reset_index()
# eda_dic = {}
# floor_list = eda_unstack.columns[3:]

# def eda_dic_maker(x) :
#     custid = x.iloc[0]
#     if custid not in eda_dic.keys() :
#         eda_dic[custid] = {}
        
#     date = x.iloc[1]
#     if date not in eda_dic[custid].keys() :
#         eda_dic[custid][date] = []
    
#     visit_floor_flg = x.iloc[3:].notnull()
#     visit_floor_list = floor_list[visit_floor_flg].tolist()
#     eda_dic[custid][date].extend(visit_floor_list)

# eda_unstack.apply(eda_dic_maker, axis=1)
# eda_list = eda_dic.copy()
# for custid in eda_dic.keys() :
#     for date in eda_dic[custid].keys() :
#         visit_floor_list = eda_dic[custid][date]
#         eda_list[custid] = []
#         for idx, floor in enumerate(visit_floor_list) :
#             if idx == 0 :
#                 movement_score = 1
#                 prep_floor = floor

#             next_floor = floor
#             movement_score += np.absolute(next_floor-prep_floor)
#             eda_list[custid].append(movement_score)
#         eda_list[custid] = np.mean(eda_list[custid])
# f = pd.DataFrame.from_dict(eda_list, orient='index').reset_index()
# f.columns = ['custid', 'mm_score']
# features['동선거리점수'] = f;

In [ ]:
X_train = pd.DataFrame({'custid': tr_train.custid.unique()})
for f in features.values() :
    X_train = pd.merge(X_train, f, on='custid', how='left')
display(X_train.head())

X_test = pd.DataFrame({'custid': tr_test.custid.unique()})
for f in features.values() :
    X_test = pd.merge(X_test, f, on='custid', how='left')
display(X_test.head())

In [ ]:
IDtest = X_test.custid;
X_train.drop(['custid'], axis=1, inplace=True)
X_test.drop(['custid'], axis=1, inplace=True)

In [ ]:
for idx, val in enumerate(features.keys()) :
    print("적용된 Feature Method{} : {}".format(idx, val))

In [ ]:
clf = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1, colsample_bytree=0.8, gamma=5, learning_rate=0.1, 
                         max_delta_step=0,  max_depth=3, min_child_weight=5, missing=None, n_estimators=100, n_jobs=1, nthread=None,
                         objective='binary:logistic', random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
                         silent=True, subsample=0.6)
# score = cross_val_score(clf, X_train, y_train.gender, cv=5, scoring='roc_auc')
# print('{}\nmean = {:.5f}\nstd = {:.5f}'.format(score, score.mean(), score.std()))

In [ ]:
pred_e = clf.fit(X_train, y_train.gender).predict_proba(X_test)[:,1]

In [ ]:
pred = (pred_a*pred_b*pred_c*pred_d*pred_e)**(1/5)
fname = 'submissions_5_20.csv'
submissions = pd.concat([IDtest, pd.Series(pred, name="gender")] ,axis=1)
submissions.to_csv(fname, index=False)